# GIS Hackathon EDA

## Load Packages

In [2]:
########## Load Packages
import warnings
warnings.simplefilter("ignore")

import importlib
import pandas as pd
import numpy as np
import re
from io import StringIO
import itertools
import os 
import time
import datetime
import geopandas as gpd
import glob

## Set Up

In [3]:
# Set Parameters 
import_bucket = "data/"

output_bucket = 'outputs/' # already created on S3

## Look Through Datasets

This Dataset gives us where refugees came from and which state they went to by year. Can track both over time. 

### 2. Refugee Admissions Report as of 30 Apr 2021.xlsx

The above gives cumulative summary of refugees by region.

In [4]:
def import_sheets_data(import_bucket, file, sheet_list, skiprow):
    '''Read in data from multiple sheets in excel files into Pandas dataframe. 
        Do some preprocessing. 
    '''
    dataframes = []
    for name in sheet_list:
        print("About to read in a file")
        df = pd.read_excel(import_bucket + file, header=[0,1,2,3], sheet_name=name, skiprows=skiprow)
        df.columns = df.columns.map(' '.join).str.strip(' ')
        df.columns = df.columns.str.replace('Unnamed:', '')
        pattern = re.compile('[^a-zA-Z]_level_[^a-zA-Z]')
        df = df.rename(columns=lambda x: pattern.sub('', x))
        d = dict(zip(df.columns[4:].values, ['OCT','NOV','DEC','JAN','FEB','MAR', 'APR', 'MAY', 'JUN', 'JUL', 'AUG', 'SEP']))
        df = df.rename(columns=d)
        df['FY'] = df.columns[0].split()[-1]
        df.columns = df.columns.str.replace('Fiscal Year', '')
        df.columns = df.columns.str.replace(r'\d+', '')
        df.columns.values[0] = "Region of Origin"
        df.columns.values[1] = "Country of Origin"
        cols = df.columns.tolist()
        value_vars = cols[4:16]
        del cols[3:16]
        df = df.melt(id_vars=cols, value_vars=value_vars, var_name="Month", value_name='ref_num', ignore_index=True)
        df['Month'] = pd.to_datetime(df.Month, format='%b').dt.month
        dataframes.append(df)
    df_full = pd.concat(dataframes, ignore_index=True, sort=False)
    df_full['Region of Origin'] = df_full['Region of Origin'].fillna(method='ffill')
    df_full.columns.values[2] = "Region Refugee Admissions Ceiling"
    df_full.columns = [x.strip() for x in df_full.columns]
    df_full['Region Refugee Admissions Ceiling'] = df_full['Region Refugee Admissions Ceiling'].fillna(method='ffill')
    return df_full

In [5]:
year_list = ([str(x) for x in list(range(2001,2022,1))])

In [6]:
monthly_ref_arr = import_sheets_data(import_bucket, "Refugee Admissions Report as of 30 Apr 2021.xlsx", year_list, 5)

About to read in a file
About to read in a file
About to read in a file
About to read in a file
About to read in a file
About to read in a file
About to read in a file
About to read in a file
About to read in a file
About to read in a file
About to read in a file
About to read in a file
About to read in a file
About to read in a file
About to read in a file
About to read in a file
About to read in a file
About to read in a file
About to read in a file
About to read in a file
About to read in a file


In [7]:
monthly_ref_arr[monthly_ref_arr['Region of Origin'] == "Europe"]

,Region of Origin,Country of Origin,Region Refugee Admissions Ceiling,FY,Month,ref_num
40,Europe,NaN,20000.0,2001,10,NaN
41,Europe,Albania,20000.0,2001,10,0.0
42,Europe,Bosnia and Herzegovina,20000.0,2001,10,822.0
43,Europe,Croatia,20000.0,2001,10,2.0
44,Europe,Greece,20000.0,2001,10,0.0
...,...,...,...,...,...,...
20059,Europe,Kazakhstan,4000.0,2021,9,0.0
20060,Europe,Moldova,4000.0,2021,9,0.0
20061,Europe,Russia,4000.0,2021,9,0.0
20062,Europe,Ukraine,4000.0,2021,9,0.0


The above provides data on where refugees came from by month by FY and includes the refugee admissions ceiling. We can calculate the extent to which regional ceilings were exceeded or not met over time and look at where refugees are coming from over time. 

Can match to other dataa above. For example, if we have 20 Ugandans in Mar. of FY 2021, we can see that X of those went to Alabama, X went to Georgia, and X went to Illinois. We wlaredy have that in the data above though. Capacity is only regional, so we can't really match that by state or city. 

In [8]:
monthly_ref_arr.to_csv('monthly_ref_arr.csv', index=False, encoding = "utf_8_sig")

In [9]:
annual_origins = monthly_ref_arr.groupby(by=["Region of Origin", "Country of Origin", "FY"], dropna=False, as_index=False).agg({'Region Refugee Admissions Ceiling':'mean', 'ref_num':'sum'})
annual_origins['FY'] = annual_origins['FY'].astype(int)
annual_origins.head()

,Region of Origin,Country of Origin,FY,Region Refugee Admissions Ceiling,ref_num
0,Africa,Angola,2001,20000.0,34.0
1,Africa,Angola,2002,22000.0,16.0
2,Africa,Angola,2003,20000.0,21.0
3,Africa,Angola,2004,30000.0,20.0
4,Africa,Angola,2005,20000.0,21.0


In [10]:
#annual_origins.to_csv('annual_origins.csv', index=False, encoding = "utf_8_sig")

In [11]:
annual_origins['FY'].value_counts()

2001    92
2016    91
2017    90
2010    85
2003    83
2011    82
2015    82
2006    81
2014    80
2009    79
2012    79
2004    78
2013    77
2018    77
2019    77
2008    77
2007    76
2002    76
2005    76
2020    67
2021    53
Name: FY, dtype: int64

### 4. Ap Refugee Data

https://data.world/associatedpress/refugee-admissions-to-us-end-fy-2018#

In [12]:
ref_arrival_cities = pd.read_csv(import_bucket + "ap_refugee_fy2018/annual_refugee_data.csv", error_bad_lines=False, encoding='utf-8')
ref_arrival_cities.columns.values[6] = "cityorigin"
ref_arrival_cities.head()

,state,year,statetotal,origin,stateorigin,city,cityorigin,annualtotal
0,Alabama,2009,187,Afghanistan,0,Mobile,0,74654
1,Alabama,2010,159,Afghanistan,13,Mobile,13,73309
2,Alabama,2011,89,Afghanistan,5,Mobile,5,56424
3,Alabama,2012,145,Afghanistan,7,Mobile,7,58238
4,Alabama,2013,129,Afghanistan,9,Mobile,9,69926


In [13]:
ref_arrival_cities[(ref_arrival_cities['state'] == "Alabama") & (ref_arrival_cities['year'] == 2018)]

,state,year,statetotal,origin,stateorigin,city,cityorigin,annualtotal
9,Alabama,2018,38,Afghanistan,0,Mobile,0,22491
19,Alabama,2018,38,Burma,1,Auburn,0,22491
29,Alabama,2018,38,Burma,1,Birmingham,1,22491
39,Alabama,2018,38,Burma,1,Dothan,0,22491
49,Alabama,2018,38,Burma,1,Mobile,0,22491
59,Alabama,2018,38,Burundi,0,Mobile,0,22491
69,Alabama,2018,38,Central African Republic,0,Mobile,0,22491
79,Alabama,2018,38,Colombia,0,Bessemer,0,22491
89,Alabama,2018,38,Colombia,0,Mobile,0,22491
99,Alabama,2018,38,Cuba,0,Birmingham,0,22491


In [14]:
ref_arrival_cities['year'].value_counts()

2009    9806
2010    9806
2011    9806
2012    9806
2013    9806
2014    9806
2015    9806
2016    9806
2017    9806
2018    9806
Name: year, dtype: int64

### 5. Affiliate Data

In [15]:
#!pip3 install tabula-py 

In [16]:
import tabula

In [17]:
dfs = tabula.io.read_pdf(import_bucket + "R&P Affiliate Directory (Updated May 2021).pdf", pages='all')
affilifates = pd.concat(dfs, ignore_index=True, sort=False)

In [18]:
affilifates = affilifates[affilifates['Agency'].notna()]
affilifates

,State,Agency,Local Agency Name,Address,City,Zip,General Phone,Alternate Phone,General Fax,General Email Mailbox,Immigration/AOR Email Mailbox,Website
1,AK,USCCB,Catholic Social Services (Alaska),3710 E. 20th Ave.,Anchorage,99508.0,907-222-7300,NaN,907-276-5539,rais@cssalaska.org,NaN,www.cssalaska.org
3,AR,LIRS,Canopy of Northwest Arkansas,"2592 N Gregg Ave, #40",Fayetteville,72703.0,479-717-7358,NaN,NaN,admin@canopynwa.org,lemine.mbareck@canopynwa.org; joanna.krause@ca...,www.canopynwa.org
5,AZ,IRC,International Rescue Committee (IRC),"4425 West Olive Avenue, Suite 400",Glendale,85302.0,602-433-2440,NaN,602-433-2881,phoenix@rescue.org,Phoenix.AOR@Rescue.org,https://www.rescue.org/phoenix
6,AZ,ECDC,Arizona Immigrant and Refugee Services,"10240 North 31st Avenue, Suite 112",Phoenix,85051.0,602-944-1821,NaN,602-944-1860,info@airsaz.org,NaN,https://airsaz.org/
7,AZ,LIRS,Lutheran Social Services of the Southwest,"2502 E. University Drive, Suite 125",Phoenix,85034.0,480-396-3795,NaN,602-248-4968,requests-comments@lss-sw.org,obabadzhanova@lss-sw.org,http://www.lss-sw.org
...,...,...,...,...,...,...,...,...,...,...,...,...
247,WA,LIRS,Lutheran Community Services Northwest,"3600 Main Street, Suite #200",Vancouver,98663.0,360-694-5624,NaN,360-694-8515,mjohnston@lcsnw.org,jcehajic@lcsnw / yparenko@lcsnw.org,www.lcsnw.org
249,WI,WR,World Relief - Fox Valley,510 E. Wisconsin Avenue,Appleton,54911.0,920-231-3600,NaN,855-728-6773,wrfoxvalley@wr.org,NaN,worldrelieffoxvalley.org
250,WI,HIAS,Jewish Social Services of Madison,6434 Enterprise Ln,Madison,53719.0,608-442-4081,NaN,608-278-7814,info@jssmadison.org,NaN,www.jssmadison.org
251,WI,USCRI,International Institute Of Wisconsin,"1110 N Old World 3rd St, Suite 420",Milwaukee,53203.0,414-225-6220,NaN,414-225-6235,info@iiwisconsin.org,NaN,https://www.iiwisconsin.org/


In [19]:
affiliates_city = affilifates[['City', 'State', 'Agency']].groupby(by=["City", "State","Agency"], dropna=False, as_index=False).size()
affiliates_city.head()

,City,State,Agency,size
0,Abilene,TX,IRC,1
1,Akron,OH,USCRI,1
2,Albany,NY,USCRI,1
3,Albuquerque,NM,LIRS,1
4,Allentown,PA,LIRS,1


In [20]:
affiliates_wide = affiliates_city.pivot(index=['City','State'], columns='Agency', values='size').reset_index()
affiliates_wide.columns.values[3] = "EMM"
affiliates_wide = affiliates_wide.fillna(0)
affiliates_wide["FY"] = "2021"
affiliates_wide = affiliates_wide[["City", "State", "FY","CWS","ECDC","EMM","HIAS","IRC","LIRS","USCCB","USCRI","WR"]]
affiliates_wide.head()

Agency,City,State,FY,CWS,ECDC,EMM,HIAS,IRC,LIRS,USCCB,USCRI,WR
0,Abilene,TX,2021,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,Akron,OH,2021,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,Albany,NY,2021,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,Albuquerque,NM,2021,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,Allentown,PA,2021,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [21]:
#affilifates.to_csv('affiliates_2021.csv', index=False, encoding = "utf_8_sig")

In [22]:
affiliate_map_time = pd.read_excel(import_bucket + "Affiliate_Map_Time.xlsx", sheet_name= "Sheet1", skiprows=0)
affiliate_map_time = affiliate_map_time.fillna(0)
affiliate_map_time.head()

,City,State,FY,CWS,ECDC,EMM,HIAS,IRC,LIRS,USCCB,USCRI,WR
0,Anchorage,Alaska,2020,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,Seattle,Washington,2020,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0
2,Tacoma,Washington,2020,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,Kent,Washington,2020,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
4,Richland,Washington,2020,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [23]:
state_codes = pd.read_csv("https://gist.githubusercontent.com/dantonnoriega/bf1acd2290e15b91e6710b6fd3be0a53/raw/11d15233327c8080c9646c7e1f23052659db251d/us-state-ansi-fips.csv", error_bad_lines=False, encoding='utf-8')
state_codes[' st'] = state_codes[' st'].astype('str')
state_codes[' st'] = state_codes[' st'].map(lambda x: x.zfill(2))
state_codes.head()

,stname,st,stusps
0,Alabama,01,AL
1,Alaska,02,AK
2,Arizona,04,AZ
3,Arkansas,05,AR
4,California,06,CA


In [24]:
affiliate_map_time = pd.merge(affiliate_map_time, state_codes, how = 'left', left_on=['State'], right_on=['stname'])
affiliate_map_time['State'] = affiliate_map_time[' stusps']
affiliate_map_time = affiliate_map_time.drop(columns=["stname", " st", ' stusps'])
affiliate_map_time.head()

,City,State,FY,CWS,ECDC,EMM,HIAS,IRC,LIRS,USCCB,USCRI,WR
0,Anchorage,AK,2020,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,Seattle,WA,2020,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0
2,Tacoma,WA,2020,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,Kent,WA,2020,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
4,Richland,WA,2020,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [25]:
affiliate_full = pd.concat([affiliates_wide,affiliate_map_time], ignore_index=True, sort=False)
affiliate_full.head()

,City,State,FY,CWS,ECDC,EMM,HIAS,IRC,LIRS,USCCB,USCRI,WR
0,Abilene,TX,2021,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,Akron,OH,2021,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,Albany,NY,2021,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,Albuquerque,NM,2021,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,Allentown,PA,2021,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [26]:
#affiliate_full.to_csv('affiliate_full_16_21s.csv', index=False, encoding = "utf_8_sig")

Merge in City Data for Mapping

In [27]:
affiliate_full[affiliate_full['City'].str.contains("Clinton")]

,City,State,FY,CWS,ECDC,EMM,HIAS,IRC,LIRS,USCCB,USCRI,WR
485,Clinton Township,MI,2018,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
664,Clinton Township,MI,2017,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
846,Clinton Township,MI,2016,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [29]:
shapefile = gpd.read_file(import_bucket + "cities-towns/citiesx020.shp")
#shapefile['NAME'] = shapefile['NAME'].str.replace("Saint", 'St.')
shapefile['for_tiger'] = shapefile['STATE_FIPS'] + shapefile['FIPS55']

In [30]:
affiliate_full['State'] = affiliate_full['State'].str.strip()
affiliate_full['City'] = affiliate_full['City'].str.strip()
affiliate_full['City']= affiliate_full['City'].str.replace("St. ", 'Saint ')
affiliate_full['City']= affiliate_full['City'].str.replace("Winston Salem", 'Winston-Salem')
affiliate_full['City']= affiliate_full['City'].str.replace("Clinton Township", 'Clinton')

affiliate_full_withgeo = pd.merge(affiliate_full, shapefile, how = 'left', left_on=['City', 'State'], right_on=['NAME', 'STATE'])

affiliate_full_withgeo['geometry'] = affiliate_full_withgeo['geometry'].astype('str')

affiliate_full_withgeo.head(3)

,City,State,FY,CWS,ECDC,EMM,HIAS,IRC,LIRS,USCCB,...,FEATURE,NAME,FIPS,FIPS55,POP,STATE_FIPS,STATE,COUNTY,geometry,for_tiger
0,Abilene,TX,2021,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,"Population 100,000 to 249,999 County Seat",Abilene,48441,01000,106654.0,48,TX,Taylor County,POINT (-99.73384094238281 32.44894027709961),4801000
1,Akron,OH,2021,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,"Population 100,000 to 249,999 County Seat",Akron,39153,01000,223019.0,39,OH,Summit County,POINT (-81.51885223388672 41.08151626586914),3901000
2,Albany,NY,2021,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,"Population 100,000 to 249,999 State Capital ...",Albany,36001,01000,101082.0,36,NY,Albany County,POINT (-73.75520324707031 42.65290069580078),3601000


In [31]:
affiliate_full_withgeo['geometry'].iloc[1]

'POINT (-81.51885223388672 41.08151626586914)'

In [32]:
affiliate_full_withgeo['geometry'] = affiliate_full_withgeo.apply(lambda x: "POINT (-73.3694744 44.5504664)" if x['City'] == 'Colchester' else x['geometry'],axis=1)

affiliate_full_withgeo['geometry'] = affiliate_full_withgeo.apply(lambda x: "POINT (-80.389192 25.8186392)" if x['City'] == 'Doral' else x['geometry'],axis=1)

affiliate_full_withgeo['geometry'] = affiliate_full_withgeo.apply(lambda x: "POINT (-122.3306402 47.4422741)" if x['City'] == 'SeaTac' else x['geometry'],axis=1)

affiliate_full_withgeo['geometry'] = affiliate_full_withgeo.apply(lambda x: "POINT (-74.0150372 40.6452228)" if ((x['City'] == 'Brooklyn') & (x['State'] == 'NY')) else x['geometry'],axis=1)

affiliate_full_withgeo['geometry'] = affiliate_full_withgeo.apply(lambda x: "POINT (-71.1363335 42.3138461)" if ((x['City'] == 'Jamaica Plain') & (x['State'] == 'MA')) else x['geometry'],axis=1)

affiliate_full_withgeo['geometry'] = affiliate_full_withgeo.apply(lambda x: "POINT (-71.0547323 42.3381569)" if x['City'] == 'South Boston' else x['geometry'],axis=1)

affiliate_full_withgeo['geometry'] = affiliate_full_withgeo.apply(lambda x: "POINT (-72.6837015 42.3380214)" if ((x['City'] == 'Florence') & (x['State'] == 'MA')) else x['geometry'],axis=1)

affiliate_full_withgeo['geometry'] = affiliate_full_withgeo.apply(lambda x: "POINT (-71.1041378 42.3138401)" if ((x['City'] == 'Roxbury') & (x['State'] == 'MA')) else x['geometry'],axis=1)






In [33]:
affiliate_full_withgeo[affiliate_full_withgeo['geometry'] == "None"]

,City,State,FY,CWS,ECDC,EMM,HIAS,IRC,LIRS,USCCB,...,FEATURE,NAME,FIPS,FIPS55,POP,STATE_FIPS,STATE,COUNTY,geometry,for_tiger


In [34]:
def get_longitude(x):
    y = x.split(" ")[1].replace("(", "")
    return y
def get_latitude(x):
    y = x.split(" ")[2].replace(")", "")
    return y

In [35]:
affiliate_full_withgeo['latitude'] = affiliate_full_withgeo['geometry'].map(lambda x: get_latitude(x))
affiliate_full_withgeo['longitude'] = affiliate_full_withgeo['geometry'].map(lambda x: get_longitude(x))

affiliate_full_withgeo.head()

,City,State,FY,CWS,ECDC,EMM,HIAS,IRC,LIRS,USCCB,...,FIPS,FIPS55,POP,STATE_FIPS,STATE,COUNTY,geometry,for_tiger,latitude,longitude
0,Abilene,TX,2021,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,48441,01000,106654.0,48,TX,Taylor County,POINT (-99.73384094238281 32.44894027709961),4801000,32.44894027709961,-99.73384094238281
1,Akron,OH,2021,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,39153,01000,223019.0,39,OH,Summit County,POINT (-81.51885223388672 41.08151626586914),3901000,41.08151626586914,-81.51885223388672
2,Albany,NY,2021,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,36001,01000,101082.0,36,NY,Albany County,POINT (-73.75520324707031 42.65290069580078),3601000,42.65290069580078,-73.75520324707031
3,Albuquerque,NM,2021,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,35001,02000,384736.0,35,NM,Bernalillo County,POINT (-106.6517639160156 35.08336639404297),3502000,35.08336639404297,-106.6517639160156
4,Allentown,PA,2021,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,42077,02000,105090.0,42,PA,Lehigh County,POINT (-75.4656982421875 40.60851287841797),4202000,40.60851287841797,-75.4656982421875


In [36]:
affiliate_full_withgeo.to_csv('affiliate_full_16_21s_geo.csv', index=False, encoding = "utf_8_sig")

### 6. Annual Survey of Refugees Data

In [37]:
asr_2018 = pd.read_csv(import_bucket + "asr_data/2018 ASR_Public_Use_File.csv", error_bad_lines=False, encoding='utf-8')
asr_2018.head()


,Unnamed: 0,hhid,qn1a,numppl,qn1b,qn1c,qn1d,qn1f,qn1g,qn1h,...,qn34b_05,ui_qn8a_annual,ui_qn10a_annual,ui_cashassist,ui_lfp,ui_emprate,ui_medicaidrma,ui_lpr,ui_school,ui_work
0,0,10000001.0,(record hh member #4 if applicable),5.0,child / stepchild / foster child / ward,NaN,8.0,female,other,eritrea,...,NaN,NaN,NaN,does not receive cash assistance,NaN,NaN,NaN,NaN,NaN,NaN
1,1,10000001.0,(record respondent name),5.0,self,now married (note: spouse need not live in hou...,42.0,female,eritrea,eritrea,...,NaN,NaN,NaN,does not receive cash assistance,not in labor force,not in labor force,individual receives rma/medicaid,already adjusted lpr status,none,not working now and never worked in past
2,2,10000001.0,(record hh member #2 if applicable),5.0,spouse (wife/husband),now married (note: spouse need not live in hou...,45.0,male,eritrea,eritrea,...,NaN,NaN,NaN,does not receive cash assistance,in labor force,employed,individual receives rma/medicaid,already adjusted lpr status,none,working now
3,3,10000001.0,(record hh member #3 if applicable),5.0,child / stepchild / foster child / ward,NaN,9.0,female,other,eritrea,...,NaN,NaN,NaN,does not receive cash assistance,NaN,NaN,NaN,NaN,NaN,NaN
4,4,10000001.0,(record hh member #5 if applicable),5.0,child / stepchild / foster child / ward,NaN,4.0,female,other,eritrea,...,NaN,NaN,NaN,does not receive cash assistance,NaN,NaN,NaN,NaN,NaN,NaN


### Allocate Cities to Rows

In [38]:
asr_2018['qn1k'].value_counts()

midwest       1473
south         1413
west          1115
northeast      840
don't know      94
refused         10
Name: qn1k, dtype: int64

In [39]:
region_state = pd.read_csv(import_bucket + "Region_State_Map.csv", error_bad_lines=False, encoding='utf-8')
region_state.head()

,State,Abbrev,Code,Region
0,Alabama,Ala.,AL,south
1,Alaska,Alaska,AK,west
2,Arizona,Ariz.,AZ,west
3,Arkansas,Ark.,AR,south
4,California,Calif.,CA,west


In [40]:
ref_arrival_cities.head()

,state,year,statetotal,origin,stateorigin,city,cityorigin,annualtotal
0,Alabama,2009,187,Afghanistan,0,Mobile,0,74654
1,Alabama,2010,159,Afghanistan,13,Mobile,13,73309
2,Alabama,2011,89,Afghanistan,5,Mobile,5,56424
3,Alabama,2012,145,Afghanistan,7,Mobile,7,58238
4,Alabama,2013,129,Afghanistan,9,Mobile,9,69926


So the first row, for example, says in 2016, there were 9 refugees from Afghanistan that went to Mobile out of the 9 Afghans that went to Alabama in 2016. 120 Refugees overall went to Alabama in 2016. 84,994 total refugees came in 2016 to the US. The refugee admissions ceiling for the Near East/South Asia region in 2016 was 38,000 refugees. 2,737 Afghani refugees came into the US in 2016. According to the census, there were 0 others born in Afghanistan in Mobile in 2016. 

So we have origin and region. We need to see what percent of each origin in a region came from each state. Then what percent of each origin in each state came from each city. 

In [41]:
st_cit_ref_arrival_perc = ref_arrival_cities.copy()

In [42]:
st_cit_ref_arrival_perc =st_cit_ref_arrival_perc[st_cit_ref_arrival_perc['city'].notna()]

In [43]:
st_cit_ref_arrival_perc = st_cit_ref_arrival_perc[st_cit_ref_arrival_perc ['year'] == 2018]
st_cit_ref_arrival_perc.head()

,state,year,statetotal,origin,stateorigin,city,cityorigin,annualtotal
9,Alabama,2018,38,Afghanistan,0,Mobile,0,22491
19,Alabama,2018,38,Burma,1,Auburn,0,22491
29,Alabama,2018,38,Burma,1,Birmingham,1,22491
39,Alabama,2018,38,Burma,1,Dothan,0,22491
49,Alabama,2018,38,Burma,1,Mobile,0,22491


In [44]:
st_cit_ref_arrival_perc = pd.merge(st_cit_ref_arrival_perc, region_state[['State','Region']], how = 'left', left_on=['state'], right_on=['State'])

st_cit_ref_arrival_perc_region = st_cit_ref_arrival_perc.groupby(by=["Region", "year",'origin'], dropna=False, as_index=False).sum()

st_cit_ref_arrival_perc_region = st_cit_ref_arrival_perc_region[['Region','year','origin','cityorigin']]

st_cit_ref_arrival_perc_region.head()

,Region,year,origin,cityorigin
0,midwest,2018,Afghanistan,158
1,midwest,2018,Angola,1
2,midwest,2018,Armenia,0
3,midwest,2018,Azerbaijan,0
4,midwest,2018,Bangladesh,0


In [45]:
st_cit_ref_arrival_perc_region = st_cit_ref_arrival_perc_region.rename(columns={'cityorigin': "regiontotal"})

In [46]:
st_cit_ref_arrival_perc = pd.merge(st_cit_ref_arrival_perc, st_cit_ref_arrival_perc_region[['Region', 'year','origin','regiontotal']], how = 'left', left_on=['Region','year','origin'], right_on=['Region','year','origin'])

st_cit_ref_arrival_perc.head()

,state,year,statetotal,origin,stateorigin,city,cityorigin,annualtotal,State,Region,regiontotal
0,Alabama,2018,38,Afghanistan,0,Mobile,0,22491,Alabama,south,224
1,Alabama,2018,38,Burma,1,Auburn,0,22491,Alabama,south,1149
2,Alabama,2018,38,Burma,1,Birmingham,1,22491,Alabama,south,1149
3,Alabama,2018,38,Burma,1,Dothan,0,22491,Alabama,south,1149
4,Alabama,2018,38,Burma,1,Mobile,0,22491,Alabama,south,1149


In [47]:
st_cit_ref_arrival_perc['city_per_state'] = st_cit_ref_arrival_perc['cityorigin']/st_cit_ref_arrival_perc['stateorigin'] 

st_cit_ref_arrival_perc['state_per_region'] = st_cit_ref_arrival_perc['stateorigin']/st_cit_ref_arrival_perc['regiontotal']

st_cit_ref_arrival_perc.head()

,state,year,statetotal,origin,stateorigin,city,cityorigin,annualtotal,State,Region,regiontotal,city_per_state,state_per_region
0,Alabama,2018,38,Afghanistan,0,Mobile,0,22491,Alabama,south,224,NaN,0.00000
1,Alabama,2018,38,Burma,1,Auburn,0,22491,Alabama,south,1149,0.0,0.00087
2,Alabama,2018,38,Burma,1,Birmingham,1,22491,Alabama,south,1149,1.0,0.00087
3,Alabama,2018,38,Burma,1,Dothan,0,22491,Alabama,south,1149,0.0,0.00087
4,Alabama,2018,38,Burma,1,Mobile,0,22491,Alabama,south,1149,0.0,0.00087


In [48]:
pd.set_option('display.max_rows', None)
st_cit_ref_arrival_perc['origin'].value_counts()

Iraq                              1598
Iran                               779
Burma                              751
Cuba                               469
Ukraine                            374
Somalia                            370
Eritrea                            364
Syria                              320
Dem. Rep. Congo                    319
El Salvador                        289
Bhutan                             283
Afghanistan                        261
Ethiopia                           261
Russia                             240
Vietnam                            239
Sudan                              212
Pakistan                           167
Liberia                            145
Moldova                            144
Burundi                            125
Colombia                           117
Rwanda                             110
Palestine                          104
Belarus                             93
China                               87
Nepal                    

In [49]:
pd.set_option('display.max_rows', 10)

In [50]:
asr_2018_for_merge = asr_2018.copy()

In [51]:
def resolve_countries(x):
    if x == "other":
        r = np.nan
    elif x == "united states":
        r = np.nan
    elif x == "don't know":
        r = np.nan
    elif x == "refused":
        r = np.nan
    else:
        r = x
    return r

In [52]:
asr_2018_for_merge['qn1g'] = asr_2018_for_merge['qn1g'].astype('str')
asr_2018_for_merge['qn1g'] = asr_2018_for_merge['qn1g'].map(lambda x: resolve_countries(x))

In [53]:
asr_2018_for_merge['qn1g'] = asr_2018_for_merge.apply(lambda x: x['qn1h'] if pd.isnull(x['qn1g']) else x['qn1g'], axis=1)

In [54]:
asr_2018_for_merge['qn1g'] = asr_2018_for_merge['qn1g'].astype('str')
asr_2018_for_merge['qn1g'] = asr_2018_for_merge['qn1g'].map(lambda x: resolve_countries(x))

In [55]:
asr_2018_for_merge = asr_2018_for_merge[(asr_2018_for_merge['qn1g'].notna()) & (asr_2018_for_merge['qn1g'] != "none")]

In [56]:
asr_2018_for_merge = asr_2018_for_merge[(asr_2018_for_merge['qn1k'].notna()) & (asr_2018_for_merge['qn1k'] != "don't know")&(asr_2018_for_merge['qn1k'] != "refused")]

In [57]:
# So we've only got Iraq, Syria, Somalia, Bhutan, Burma, DRC, Iran, Eritrea, Nepal, Thailand, Ukraine, and El Salvador

In [58]:
asr_2018_for_merge['qn1g'] = asr_2018_for_merge['qn1g'].map(lambda x: x.title())

In [59]:
asr_2018_for_merge['qn1g']  = asr_2018_for_merge['qn1g'] .str.replace("Democratic Republic Of The Congo", 'Dem. Rep. Congo')

In [60]:
#asr_2018[['qn1h','qn1k']].head()
asr_2018_for_merge['qn1g'].value_counts()

Iraq           1533
Syria           652
Bhutan          394
Burma           372
Somalia         372
               ... 
Eritrea         143
Thailand        129
Nepal           127
Ukraine         103
El Salvador      98
Name: qn1g, Length: 12, dtype: int64

In [61]:
asr_2018_for_merge['qn1k'].value_counts()

midwest      1397
south        1249
west         1003
northeast     758
Name: qn1k, dtype: int64

In [62]:
elsalv_check = asr_2018_for_merge[asr_2018_for_merge['qn1g'] == "El Salvador"]
elsalv_check['qn1k'].value_counts()

south        38
northeast    32
west         22
midwest       6
Name: qn1k, dtype: int64

In [63]:
perc_elsalv_check = st_cit_ref_arrival_perc[(st_cit_ref_arrival_perc['origin'] == "El Salvador") &(st_cit_ref_arrival_perc['Region'] == "south")]
perc_elsalv_check.sort_values(["state_per_region", "city_per_state"], ascending = (False, False)).head()

,state,year,statetotal,origin,stateorigin,city,cityorigin,annualtotal,State,Region,regiontotal,city_per_state,state_per_region
3931,Maryland,2018,465,El Salvador,83,Silver Spring,27,22491,Maryland,south,377,0.325301,0.220159
3929,Maryland,2018,465,El Salvador,83,Riverdale,22,22491,Maryland,south,377,0.265060,0.220159
3923,Maryland,2018,465,El Salvador,83,Baltimore,20,22491,Maryland,south,377,0.240964,0.220159
3927,Maryland,2018,465,El Salvador,83,Hyattsville,11,22491,Maryland,south,377,0.132530,0.220159
3924,Maryland,2018,465,El Salvador,83,College Park,1,22491,Maryland,south,377,0.012048,0.220159


21.7% of El Salvadorian refugees live in Maryland, and 32.5% of those live in Silver Spring. So of the 38 we have in the south, about 8 should be in Maryland, and about 3 should be in Silver Spring. This is what we need to set up. 

In [64]:
asr_origin_region_counts = asr_2018_for_merge[['qn1g','qn1k']].groupby(by=['qn1g','qn1k'], dropna=False, as_index=False).size()

In [65]:
st_cit_ref_arrival_perc = pd.merge(st_cit_ref_arrival_perc, asr_origin_region_counts, how = 'left', left_on=['Region','origin'], right_on=['qn1k','qn1g'])

st_cit_ref_arrival_perc.sort_values(["state_per_region", "city_per_state"], ascending = (False, False)).head()

,state,year,statetotal,origin,stateorigin,city,cityorigin,annualtotal,State,Region,regiontotal,city_per_state,state_per_region,qn1g,qn1k,size
76,Arizona,2018,1000,Algeria,1,Phoenix,1,22491,Arizona,west,1,1.0,1.0,NaN,NaN,NaN
107,Arizona,2018,1000,Congo,1,Phoenix,1,22491,Arizona,west,1,1.0,1.0,NaN,NaN,NaN
189,Arizona,2018,1000,Jordan,1,Tucson,1,22491,Arizona,west,1,1.0,1.0,NaN,NaN,NaN
196,Arizona,2018,1000,Liberia,1,Phoenix,1,22491,Arizona,west,1,1.0,1.0,NaN,NaN,NaN
347,California,2018,1368,Azerbaijan,2,Walnut Creek,2,22491,California,west,2,1.0,1.0,NaN,NaN,NaN


In [66]:
st_cit_ref_arrival_perc['state_total'] = round(st_cit_ref_arrival_perc['state_per_region']* st_cit_ref_arrival_perc['size'])

st_cit_ref_arrival_perc['city_total'] = round(st_cit_ref_arrival_perc['city_per_state']* st_cit_ref_arrival_perc['state_total'])

st_cit_ref_arrival_perc['city_per_state_per_region'] = st_cit_ref_arrival_perc['city_per_state']* st_cit_ref_arrival_perc['state_per_region']

st_cit_ref_arrival_perc.sort_values(["state_total", "city_total"], ascending = (False, False))

,state,year,statetotal,origin,stateorigin,city,cityorigin,annualtotal,State,Region,regiontotal,city_per_state,state_per_region,qn1g,qn1k,size,state_total,city_total,city_per_state_per_region
8482,Texas,2018,1697,Iraq,25,San Antonio,9,22491,Texas,south,40,0.36,0.625,Iraq,south,412.0,258.0,93.0,0.225
8425,Texas,2018,1697,Iraq,25,Dallas,7,22491,Texas,south,40,0.28,0.625,Iraq,south,412.0,258.0,72.0,0.175
8443,Texas,2018,1697,Iraq,25,Houston,6,22491,Texas,south,40,0.24,0.625,Iraq,south,412.0,258.0,62.0,0.150
8405,Texas,2018,1697,Iraq,25,Austin,1,22491,Texas,south,40,0.04,0.625,Iraq,south,412.0,258.0,10.0,0.025
8489,Texas,2018,1697,Iraq,25,Texas City,1,22491,Texas,south,40,0.04,0.625,Iraq,south,412.0,258.0,10.0,0.025
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9766,Wisconsin,2018,404,Uganda,0,Oshkosh,0,22491,Wisconsin,midwest,8,NaN,0.000,NaN,NaN,NaN,NaN,NaN,NaN
9770,Wisconsin,2018,404,Vietnam,0,Hartford,0,22491,Wisconsin,midwest,11,NaN,0.000,NaN,NaN,NaN,NaN,NaN,NaN
9771,Wisconsin,2018,404,Vietnam,0,Madison,0,22491,Wisconsin,midwest,11,NaN,0.000,NaN,NaN,NaN,NaN,NaN,NaN
9772,Wisconsin,2018,404,Vietnam,0,Milwaukee,0,22491,Wisconsin,midwest,11,NaN,0.000,NaN,NaN,NaN,NaN,NaN,NaN


In [67]:
st_cit_ref_arrival_perc['city-state'] = st_cit_ref_arrival_perc[['city', 'state']].agg('-'.join, axis=1)

### Expand Grid 

In [68]:
## Create a dataframe with every combination of city and origin
index = pd.MultiIndex.from_product([st_cit_ref_arrival_perc['city-state'], st_cit_ref_arrival_perc['origin']], names = ["city-state", "origin"])

expand_city = pd.DataFrame(index = index).reset_index()
expand_city 

,city-state,origin
0,Mobile-Alabama,Afghanistan
1,Mobile-Alabama,Burma
2,Mobile-Alabama,Burma
3,Mobile-Alabama,Burma
4,Mobile-Alabama,Burma
...,...,...
95550620,Cheyenne-Wyoming,Vietnam
95550621,Cheyenne-Wyoming,Vietnam
95550622,Cheyenne-Wyoming,Vietnam
95550623,Cheyenne-Wyoming,Vietnam


In [69]:
expand_city  = expand_city.drop_duplicates()
expand_city.head()

,city-state,origin
0,Mobile-Alabama,Afghanistan
1,Mobile-Alabama,Burma
5,Mobile-Alabama,Burundi
6,Mobile-Alabama,Central African Republic
7,Mobile-Alabama,Colombia


In [70]:
st_cit_ref_arrival_perc = st_cit_ref_arrival_perc.sort_values('cityorigin', ascending=False).drop_duplicates(subset=['city-state', 'origin'], keep='first')

In [71]:
st_cit_ref_arrival_perc_for_merge = pd.merge(st_cit_ref_arrival_perc, expand_city, how = 'outer', on=['city-state','origin']).drop_duplicates()

In [72]:
st_cit_ref_arrival_perc_for_merge['city_per_state_per_region'] = st_cit_ref_arrival_perc_for_merge['city_per_state_per_region'].fillna(0)

In [73]:
asr_2018_for_merge_origin_reg = asr_2018_for_merge[['qn1g','qn1k']].drop_duplicates()
asr_2018_for_merge_origin_reg.head()

,qn1g,qn1k
0,Eritrea,northeast
6,Eritrea,south
14,Iran,west
19,Ukraine,northeast
29,Ukraine,west


In [74]:
asr_2018_for_merge_origin_reg['origin-region'] = asr_2018_for_merge_origin_reg[['qn1g', 'qn1k']].agg('-'.join, axis=1)
asr_2018_for_merge_origin_reg.head()

,qn1g,qn1k,origin-region
0,Eritrea,northeast,Eritrea-northeast
6,Eritrea,south,Eritrea-south
14,Iran,west,Iran-west
19,Ukraine,northeast,Ukraine-northeast
29,Ukraine,west,Ukraine-west


In [75]:
all_weights = {}
for i in list(asr_2018_for_merge_origin_reg['origin-region']):
    
    st_df = st_cit_ref_arrival_perc_for_merge[(st_cit_ref_arrival_perc_for_merge['origin'] == i.split("-")[0]) & (st_cit_ref_arrival_perc_for_merge['Region'] == i.split("-")[1])]
    
    weight_list = {}

    weight_list[i]=st_df['city_per_state_per_region'].to_list()

    all_weights.update(weight_list)

In [76]:
asr_2018_for_merge['origin-region'] = asr_2018_for_merge[['qn1g', 'qn1k']].agg('-'.join, axis=1)
asr_2018_for_merge.head()

list_category = asr_2018_for_merge_origin_reg['origin-region'].unique().tolist()

asr_2018_for_merge['city'] = np.nan

d = all_weights

full_survey_cities = []

for val in list_category:
    asr_df = asr_2018_for_merge[(asr_2018_for_merge['qn1g'] == val.split("-")[0]) & (asr_2018_for_merge['qn1k'] == val.split("-")[1])]
    
    city_list = st_cit_ref_arrival_perc[(st_cit_ref_arrival_perc['origin'] == val.split("-")[0]) & (st_cit_ref_arrival_perc['Region'] == val.split("-")[1])]['city-state'].unique().tolist()
    
    sz = ((asr_2018_for_merge['qn1g'] == val.split("-")[0]) & (asr_2018_for_merge['qn1k'] == val.split("-")[1])).sum() # find the size for array to create
    # use `loc` to select places you want to replace
    p = d[val]
    p = np.asarray(p).astype('float64')
    p /= p.sum()
    
    np.random.seed(0)
    
    try:
        asr_df.loc[asr_df["origin-region"] == val,'city'] = np.random.choice(city_list, sz, p=p)
        full_survey_cities.append(asr_df)
    except:
        pass

In [77]:
asr_w_cities = pd.concat(full_survey_cities, ignore_index=True, sort=False)
asr_w_cities['state'] = asr_w_cities['city'].map(lambda x: x.split("-")[1])
asr_w_cities['city'] = asr_w_cities['city'].map(lambda x: x.split("-")[0])
asr_w_cities

,Unnamed: 0,hhid,qn1a,numppl,qn1b,qn1c,qn1d,qn1f,qn1g,qn1h,...,ui_cashassist,ui_lfp,ui_emprate,ui_medicaidrma,ui_lpr,ui_school,ui_work,origin-region,city,state
0,0,10000001.0,(record hh member #4 if applicable),5.0,child / stepchild / foster child / ward,NaN,8.0,female,Eritrea,eritrea,...,does not receive cash assistance,NaN,NaN,NaN,NaN,NaN,NaN,Eritrea-northeast,Rochester,New York
1,1,10000001.0,(record respondent name),5.0,self,now married (note: spouse need not live in hou...,42.0,female,Eritrea,eritrea,...,does not receive cash assistance,not in labor force,not in labor force,individual receives rma/medicaid,already adjusted lpr status,none,not working now and never worked in past,Eritrea-northeast,Jersey City,New Jersey
2,2,10000001.0,(record hh member #2 if applicable),5.0,spouse (wife/husband),now married (note: spouse need not live in hou...,45.0,male,Eritrea,eritrea,...,does not receive cash assistance,in labor force,employed,individual receives rma/medicaid,already adjusted lpr status,none,working now,Eritrea-northeast,Providence,Rhode Island
3,3,10000001.0,(record hh member #3 if applicable),5.0,child / stepchild / foster child / ward,NaN,9.0,female,Eritrea,eritrea,...,does not receive cash assistance,NaN,NaN,NaN,NaN,NaN,NaN,Eritrea-northeast,Rochester,New York
4,4,10000001.0,(record hh member #5 if applicable),5.0,child / stepchild / foster child / ward,NaN,4.0,female,Eritrea,eritrea,...,does not receive cash assistance,NaN,NaN,NaN,NaN,NaN,NaN,Eritrea-northeast,Syracuse,New York
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4265,1795,10000842.0,(record hh member #4 if applicable),5.0,sibling / stepsister / stepbrother,never married,25.0,female,El Salvador,other,...,receives cash assistance,NaN,NaN,NaN,NaN,NaN,NaN,El Salvador-midwest,Sioux Falls,South Dakota
4266,1796,10000842.0,(record hh member #3 if applicable),5.0,parent / stepparent / foster parent / guardian,now married (note: spouse need not live in hou...,40.0,female,El Salvador,other,...,receives cash assistance,NaN,NaN,NaN,NaN,NaN,NaN,El Salvador-midwest,South Bend,Indiana
4267,4970,10002327.0,(record hh member #2 if applicable),4.0,parent / stepparent / foster parent / guardian,now married (note: spouse need not live in hou...,50.0,female,El Salvador,other,...,receives cash assistance,NaN,NaN,NaN,NaN,NaN,NaN,El Salvador-midwest,Chicago,Illinois
4268,4971,10002327.0,(record hh member #4 if applicable),4.0,sibling / stepsister / stepbrother,never married,26.0,male,El Salvador,other,...,receives cash assistance,NaN,NaN,NaN,NaN,NaN,NaN,El Salvador-midwest,St. Paul,Minnesota


In [1374]:
#asr_w_cities.to_csv('asr_2018_w_cities.csv', index=False, encoding = "utf_8_sig")

i.	Labor participation status, employment status
ii.	Medicaid
iii.	LPR status
iv.	School/education 
v.	Ages at arrival 
vi.	Cash assistance status 
vii.	Public assistance receipt 
viii.	Source of income
ix.	Annual earnings (BIG)
x.	Work status 
xi.	Vars noting which of the 9 agencies they are with (BIG)
xii.	Country of origin 	

Don' think there is any settled to geo though???

### 9. Food Access Data

In [78]:
grocery_stores = pd.read_excel(import_bucket + "Food Access Research Atlas.xlsx", sheet_name = "Food Access Research Atlas Data", skiprows = 0)
grocery_stores.head()


,CensusTract,State,County,LILATracts_1And10,LILATracts_halfAnd10,LILATracts_1And20,LILATracts_Vehicle,Urban,Rural,LA1and10,...,lapop20,lapop20share,lalowi20,lalowi20share,lakids20,lakids20share,laseniors20,laseniors20share,lahunv20,lahunv20share
0,1001020100,AL,Autauga,0,0,0,0,1,0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1001020200,AL,Autauga,0,0,0,0,1,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1001020300,AL,Autauga,0,0,0,0,1,0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1001020400,AL,Autauga,0,0,0,0,1,0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1001020500,AL,Autauga,0,0,0,0,1,0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [79]:
farmers_markets = pd.read_excel(import_bucket + "usda-local-foods-locations/SNAP-Authorized-FMs.xlsx", sheet_name = "Sheet1", skiprows = 4)
farmers_markets.head()


,Store Name,Street Number,Street Name,Addl Address,City,State,Zip,County Code,County Name
0,Anchor Point Saterday Farmers Market,70961,Sarah Ln,NaN,Anchor Point,AK,99556,122,KENAI PENINSULA
1,Anchorage Farmers Market,1420,Cordova St,NaN,Anchorage,AK,99501,20,ANCHORAGE
2,Black Bear Farms,12501,E Marsh Rd,NaN,Palmer,AK,99645,170,MATANUSKA SUSITNA
3,Cool Cache Farms,47110,Autumn Rd,NaN,Kenai,AK,99611,122,KENAI PENINSULA
4,Dandelion Acres,4412,N Dogwood Rd,NaN,Kenai,AK,99611,122,KENAI PENINSULA


### 9. Census Data

There are 5 data sets available through CensusData including the classic Decennial Census as well as 4 different American Community Survey (ACS) estimates.

The ACS started in 2005 and replaced the long form Census that used to be sent every ten years to a subset of households along with the traditional Census. Instead, the ACS is conducted on a rolling basis and sent to roughly 3.5 million households each month. This subset is then used to create estimates of the entire population. The questions on the ACS are also more in-depth and include topics like education, employment and internet access.

The Five Data Sets

- ACS 1-year estimates (2012–2018) For areas with populations 65,000+, most frequently updated but with the lowest “resolution” since it excludes areas with low population and has the smallest sample size

- ACS 1-year supplemental estimates (2014–2017) Supplemental dataset that focuses on lower population areas of 20,000+

- ACS 3-year estimates (2010–2012 to 2011–2013) For areas with populations 20,000+, very much the middle ground between the 1 and 5 years. Currently discounted by the Census Bureau but old versions can still be accessed.

- ACS 5-year estimates (2005–2009 to 2014–2018) Data for all areas, highest resolution and largest sample size but the least current

- Census 2010 Summary File 1 Counts every resident of the US, updated every 10 years.

Why don’t we use the more recent ACS for everything?

The decennial Census publishes population counts for every ‘Census block’. The ACS has more recent and more detailed statistics, but we have to zoom out one level to the ‘block group’. For other data points (such as language and national ancestry) we must zoom out again to the ‘tract’ level.

Download Census shapefiles

You can view map data in GIS software. The leading free/open source application is called QGIS: qgis.org

You can download 2010 blocks or 2018 block groups as shapefiles from
census.gov/cgi-bin/geo/shapefiles/index.php


In [205]:
outfile='zbp2017.json'

#### Getting at the data we want

In [81]:
import time, json
import requests

In [206]:
keyfile='census_key.txt'

In [208]:
#Read api key in from file
with open(keyfile) as key:
    api_key=key.read().strip()

ACS 1 Year

The American Community Survey (ACS) is an ongoing survey that provides data every year -- giving communities the current information they need to plan investments and services. The ACS covers a broad range of topics about social, economic, demographic, and housing characteristics of the U.S. population. Much of the ACS data provided on the Census Bureau's Web site are available separately by age group, race, Hispanic origin, and sex.

Detailed Tables, Subject Tables, Data Profiles, Comparison Profiles and Selected Population Profiles are available for the nation, all 50 states, the District of Columbia, Puerto Rico, every congressional district, every metropolitan area, and all counties and places with populations of 65,000 or more.

#### Get field groups

In [84]:
#acs1_vars = pd.read_html('https://api.census.gov/data/2019/acs/acs1/variables.html')

In [85]:
#acs1_vars = acs1_vars[0]
#acs1_vars.head()

In [86]:
#acs1_groups = acs1_vars[['Concept', 'Group']]
#acs1_groups = acs1_groups.drop_duplicates()
#acs1_groups['Group_app'] = acs1_groups['Group'].map(lambda x: str(x)[0:6])
#acs1_groups = acs1_groups[['Concept', 'Group_app']]
#acs1_groups = acs1_groups.drop_duplicates(subset=['Group_app'])

In [87]:
#acs1_groups.to_csv('acs1_groups.csv', index=False, encoding = "utf_8_sig")

### Pull Vars

In [88]:
#Set variables
#year='2018'
#dsource='acs'
#dname='acs1'
#cols='NAME,group(B01003)'
#keyfile='census_key.txt'

In [278]:
def retreive_census(year, dsource,dname,cols,keyfile):
    base_url = f'https://api.census.gov/data/{year}/{dsource}/{dname}'
    #Retrieve data, write output when finished
    data_url = f'{base_url}?get={cols}&key={api_key}'
    response=requests.get(data_url)
    outdata=response.json()
    with open(outfile, 'w') as f:
        json.dump(outdata, f)
    print('Data dumped to json file')
    print('File has', len(outdata), 'records')
    #Load data from output file, convert to dataframe, perform some basic operations
    with open(outfile, 'r') as f:
        data=json.load(f)
    df=pd.DataFrame(data[1:], columns=data[0])
    return df

In [279]:
#Set variables
#year='2015'
#dsource='acs'
#dname='acs1'
#cols='NAME,group(B05006)'
#keyfile='census_key.txt'
#foor = 'county:*'

In [280]:
def retreive_census_2(year, dsource,foor, dname,cols,keyfile):
    base_url = f'https://api.census.gov/data/{year}/{dsource}/{dname}'
    #Retrieve data, write output when finished
    data_url = f'{base_url}?get={cols}&for={foor}&key={api_key}'
    response=requests.get(data_url)
    outdata=response.json()
    with open(outfile, 'w') as f:
        json.dump(outdata, f)
    print('Data dumped to json file')
    print('File has', len(outdata), 'records')
    #Load data from output file, convert to dataframe, perform some basic operations
    with open(outfile, 'r') as f:
        data=json.load(f)
    df=pd.DataFrame(data[1:], columns=data[0])
    return df

In [92]:
#base_url = f'https://api.census.gov/data/{year}/{dsource}/{dname}'
#Retrieve data, write output when finished
#data_url = f'{base_url}?get={cols}&for={foor}&key={api_key}'
#data_url


Load in

In [281]:
#foreign_born_pop = retreive_census_2(year, dsource,foor,dname,cols,keyfile)
#foreign_born_pop = foreign_born_pop.iloc[:, :-1]
#foreign_born_pop.head(3)

Data dumped to json file
File has 631 records


,NAME,B05006_001E,B05006_001EA,B05006_001M,B05006_001MA,B05006_002E,B05006_002EA,B05006_002M,B05006_002MA,B05006_003E,...,B05006_161EA,B05006_161M,B05006_161MA,B05006_162E,B05006_162EA,B05006_162M,B05006_162MA,GEO_ID,NAME,state
0,"Nashua city, New Hampshire",None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,1600000US3350260,"Nashua city, New Hampshire",33
1,"Clifton city, New Jersey",None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,1600000US3413690,"Clifton city, New Jersey",34
2,"Union City city, New Jersey",None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,1600000US3474630,"Union City city, New Jersey",34
3,"Paterson city, New Jersey",None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,1600000US3457000,"Paterson city, New Jersey",34
4,"Elizabeth city, New Jersey",None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,1600000US3421000,"Elizabeth city, New Jersey",34
5,"Camden city, New Jersey",None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,1600000US3410000,"Camden city, New Jersey",34
6,"Trenton city, New Jersey",None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,1600000US3474000,"Trenton city, New Jersey",34
7,"Spring Valley CDP, Nevada",None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,1600000US3268585,"Spring Valley CDP, Nevada",32
8,"East Orange city, New Jersey",None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,1600000US3419390,"East Orange city, New Jersey",34
9,"Billings city, Montana",None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,1600000US3006550,"Billings city, Montana",30


In [94]:
#foreign_born_pop.columns

In [95]:
#cols = foreign_born_pop.columns.tolist()
#indices = [0, 649, 651, 652]
#id_cols= [cols[i] for i in indices]
#value_vars = cols[1:649]
#foreign_born_pop = foreign_born_pop.melt(id_vars=id_cols, value_vars=value_vars, var_name="Variable", value_name='value', ignore_index=True)
#foreign_born_pop.head()

In [96]:
#cols[slice(0,None,673)]

In [97]:
#cols[1:673]

In [98]:
def mark_lable(x):
    if x.endswith('E') == True:
        r = 'Estimate'
    elif x.endswith('EA') == True:
        r = 'Estimate Annotation'
    elif x.endswith('M') == True:
        r = 'Margin of Error'
    elif x.endswith('MA') == True:
        r = 'Margin of Error Annotation'
    else:
        r = "Error"
    return r     

In [99]:
#foreign_born_pop['Variable_Type'] = foreign_born_pop['Variable'].apply(lambda x: mark_lable(x))

In [100]:
#full_5006 = acs1_vars[acs1_vars['Name'].str.contains('B05006')] 
#full_5006['country'] = full_5006['Label'].map(lambda x: x.split("!!")[-1].replace(':', ''))

In [101]:
#foreign_born_pop = pd.merge(foreign_born_pop, full_5006[['Name', 'country']], how = 'left', left_on='Variable', right_on='Name')
#foreign_born_pop["value"] = pd.to_numeric(foreign_born_pop["value"])
#del foreign_born_pop['Name']

Test

In [102]:
#pd.set_option('display.max_rows', 100)
#foreign_born_pop[(foreign_born_pop['Variable'] == 'B05006_049E') & (pd.notna(foreign_born_pop['value']))]

In [103]:
#pd.set_option('display.max_rows', None)
#foreign_born_pop['NAME'].value_counts()

In [104]:
#foreign_born_pop.to_csv('foreign_born_pop_2016.csv', index=False, encoding = "utf_8_sig")

In [105]:
filepath = "foreign_born/"

In [106]:
def geo_id(x):
    y = x.split("US")[1]
    return y

In [107]:
def import_new(import_bucket, filepath, skiprow, aff_type):
    """
    Used to load in data
    """
    
    filelist = glob.glob(import_bucket + filepath + aff_type)
    dataframes = []
    
    for file in filelist:
        print("About to read in a file")
        df = pd.read_csv(file, skiprows=skiprow)
        df['FY'] = file.split("/")[2].split("_")[3].split(".")[0]
        df['for_tiger'] = df['GEO_ID'].map(lambda x: geo_id(x))
        dataframes.append(df)
    df_full = pd.concat(dataframes, ignore_index=True, sort=False)
    return df_full

In [108]:
foreign_born_full = import_new(import_bucket, filepath, 0, "*.csv")

About to read in a file
About to read in a file
About to read in a file
About to read in a file


In [109]:
foreign_born_full.head()

,NAME,GEO_ID,Variable,value,Variable_Type,country,FY,for_tiger
0,United States,0100000US,B05006_001E,44525458.0,Estimate,Total,2017,
1,South Region,0200000US3,B05006_001E,14900362.0,Estimate,Total,2017,3
2,West Region,0200000US4,B05006_001E,15127240.0,Estimate,Total,2017,4
3,Northeast Region,0200000US1,B05006_001E,9479629.0,Estimate,Total,2017,1
4,Midwest Region,0200000US2,B05006_001E,5018227.0,Estimate,Total,2017,2


In [111]:
#foreign_born_full.to_csv('foreign_born_pop_full.csv', index=False, encoding = "utf_8_sig")

## City-Level Merges 

1. Merge Regional Refugee Ceiling and total by origin by FY Numbers into city ref numbers.

Regional Ceilings and OVerall Origin to City-Level and State-Level by Origin

In [112]:
ref_arrival_cities.head(5)

,state,year,statetotal,origin,stateorigin,city,cityorigin,annualtotal
0,Alabama,2009,187,Afghanistan,0,Mobile,0,74654
1,Alabama,2010,159,Afghanistan,13,Mobile,13,73309
2,Alabama,2011,89,Afghanistan,5,Mobile,5,56424
3,Alabama,2012,145,Afghanistan,7,Mobile,7,58238
4,Alabama,2013,129,Afghanistan,9,Mobile,9,69926


In [113]:
ref_arrival_cities[(ref_arrival_cities['origin'] == "Botswana") & (ref_arrival_cities['year'] == 2017)  & (ref_arrival_cities['city'] == "Boise") ]


,state,year,statetotal,origin,stateorigin,city,cityorigin,annualtotal
27748,Idaho,2017,628,Botswana,1,Boise,1,53716


In [114]:
ref_arrival_cities.shape

(98060, 8)

In [115]:
ref_arr_cities_with_ceilings = pd.merge(ref_arrival_cities, annual_origins, how = 'left', left_on=['origin', "year"], right_on=['Country of Origin', 'FY'])
ref_arr_cities_with_ceilings.head()

,state,year,statetotal,origin,stateorigin,city,cityorigin,annualtotal,Region of Origin,Country of Origin,FY,Region Refugee Admissions Ceiling,ref_num
0,Alabama,2009,187,Afghanistan,0,Mobile,0,74654,Near East/South Asia,Afghanistan,2009.0,39500.0,349.0
1,Alabama,2010,159,Afghanistan,13,Mobile,13,73309,Near East/South Asia,Afghanistan,2010.0,38000.0,515.0
2,Alabama,2011,89,Afghanistan,5,Mobile,5,56424,Near East/South Asia,Afghanistan,2011.0,35500.0,428.0
3,Alabama,2012,145,Afghanistan,7,Mobile,7,58238,Near East/South Asia,Afghanistan,2012.0,35500.0,481.0
4,Alabama,2013,129,Afghanistan,9,Mobile,9,69926,Near East/South Asia,Afghanistan,2013.0,32400.0,661.0


In [116]:
ref_arr_cities_with_ceilings.to_csv('ref_arr_cities_with_ceilings.csv', index=False, encoding = "utf_8_sig")

2. Merge in Census Foreign Born Pop Data 

In [117]:
def geo_id(x):
    y = x.split("US")[1]
    if len(y) == 0:
        r = 'Country'
    elif len(y) == 1:
        r = 'Region'
    elif len(y) == 2:
        r = 'State'
    elif len(y) == 5:
        r = 'County or Micro Area'
    elif len(y) == 10: # Or 7 for capital?
        r = 'City Or Township OR Borrough'
    elif len(y) == 7:
        r = 'City'
    elif len(y) == 12:
        r = 'Village'
    elif len(y) == 5:
        r = 'Micro Area'
    elif len(y) == 4:
        r = 'Congressional District'
    elif len(y) == 3:
        r = 'Metro Region'
    else:
        r = "Error"
    return r     

In [118]:
def geo_split(x):
    y = x.split("US")[1]
    if len(y) == 0:
        r = 'Country'
    elif len(y) == 1:
        r = 'Region'
    elif len(y) == 2:
        r = 'State'
    elif len(y) == 5:
        r = 'County or Micro Area'
    elif len(y) == 10: # Or 7 for capital?
        r = 'City Or Township OR Borrough'
    elif len(y) == 7:
        r = 'City'
    elif len(y) == 12:
        r = 'Village'
    elif len(y) == 5:
        r = 'Micro Area'
    elif len(y) == 4:
        r = 'Congressional District'
    elif len(y) == 3:
        r = 'Metro Region'
    else:
        r = "Error"
    return r     

In [119]:
def first_name(x):
    y = x.split(",")[0]
    return y 

In [120]:
def get_state(x):
    y = x.split("US")[1]
    try: 
        y = y[0:2]
    except: 
        y = "Above"
    return y 

In [121]:
foreign_born_full['GEO_Type'] = foreign_born_full['GEO_ID'].map(lambda x: geo_id(x))

In [122]:
foreign_born_full['State_Code'] = foreign_born_full['GEO_ID'].map(lambda x: get_state(x))

In [123]:
state_codes = pd.read_csv("https://gist.githubusercontent.com/dantonnoriega/bf1acd2290e15b91e6710b6fd3be0a53/raw/11d15233327c8080c9646c7e1f23052659db251d/us-state-ansi-fips.csv", error_bad_lines=False, encoding='utf-8')
state_codes[' st'] = state_codes[' st'].astype('str')
state_codes[' st'] = state_codes[' st'].map(lambda x: x.zfill(2))
state_codes.head()

,stname,st,stusps
0,Alabama,01,AL
1,Alaska,02,AK
2,Arizona,04,AZ
3,Arkansas,05,AR
4,California,06,CA


In [124]:
foreign_born_full = pd.merge(foreign_born_full, state_codes, how = 'left', left_on=["State_Code"], right_on=[' st'])

In [125]:
foreign_born_full['NAME_First'] = foreign_born_full['NAME'].map(lambda x: first_name(x))

In [126]:
foreign_born_full['NAME_First'] = foreign_born_full['NAME_First'].str.replace(' city', '')

In [127]:
foreign_born_full['value'].fillna(0, inplace=True)

In [128]:
foreign_born_full.head()

,NAME,GEO_ID,Variable,value,Variable_Type,country,FY,for_tiger,GEO_Type,State_Code,stname,st,stusps,NAME_First
0,United States,0100000US,B05006_001E,44525458.0,Estimate,Total,2017,,Country,,NaN,NaN,NaN,United States
1,South Region,0200000US3,B05006_001E,14900362.0,Estimate,Total,2017,3,Region,3,NaN,NaN,NaN,South Region
2,West Region,0200000US4,B05006_001E,15127240.0,Estimate,Total,2017,4,Region,4,NaN,NaN,NaN,West Region
3,Northeast Region,0200000US1,B05006_001E,9479629.0,Estimate,Total,2017,1,Region,1,NaN,NaN,NaN,Northeast Region
4,Midwest Region,0200000US2,B05006_001E,5018227.0,Estimate,Total,2017,2,Region,2,NaN,NaN,NaN,Midwest Region


In [129]:
#ccc_exp = ref_arr_cities_with_ceilings.drop_duplicates(subset='city', keep='first')
#ccc_exp = ccc_exp[['city']]
#ccc_exp.to_csv('ccc_exp.csv', index=False, encoding = "utf_8_sig")

In [130]:
foreign_born_full_urban = foreign_born_full[(foreign_born_full['GEO_Type'] == "City") | (foreign_born_full['GEO_Type'] == "Micro Area") | (foreign_born_full['GEO_Type'] == "Village") | (foreign_born_full['GEO_Type'] == "City Or Township OR Borrough")]

In [131]:
foreign_born_full_urban['NAME_First'].value_counts()

Springfield                           20928
Portland                              18312
Bloomington                           18312
Lawrence                              18312
Jackson                               18312
                                      ...  
Evanston township                       648
Hueneme Elementary School District      648
South Whittier CDP                      648
Papillion-La Vista Public Schools       648
Palm Harbor CDP                         648
Name: NAME_First, Length: 4155, dtype: int64

In [132]:
#fbf_exp = foreign_born_full.drop_duplicates(subset='NAME_First', keep='first')
#fbf_exp = fbf_exp[['NAME_First', 'GEO_Type']]
#fbf_exp.to_csv('fbf_exp.csv', index=False, encoding = "utf_8_sig")

In [133]:
ref_arr_cities_with_ceilings['city'].value_counts()

Portland        600
Denver          550
Glendale        530
Rochester       500
Phoenix         480
               ... 
East Walpole     10
Dudley           10
Dedham           10
Charlestown      10
Cheyenne         10
Name: city, Length: 2498, dtype: int64

In [134]:
ref_arr_cities_with_ceilings['year'] = ref_arr_cities_with_ceilings['year'].astype(str)

In [135]:
ref_arr_cities_with_ceilings['city'] = ref_arr_cities_with_ceilings['city'].astype(str)

In [136]:
ref_arr_cities_with_ceilings.columns

Index(['state', 'year', 'statetotal', 'origin', 'stateorigin', 'city',
       'cityorigin', 'annualtotal', 'Region of Origin', 'Country of Origin',
       'FY', 'Region Refugee Admissions Ceiling', 'ref_num'],
      dtype='object')

In [137]:
foreign_born_full.columns

Index(['NAME', 'GEO_ID', 'Variable', 'value', 'Variable_Type', 'country', 'FY',
       'for_tiger', 'GEO_Type', 'State_Code', 'stname', ' st', ' stusps',
       'NAME_First'],
      dtype='object')

Merge via shapefile codes first (GEO)

In [138]:
shapefile = gpd.read_file(import_bucket + "cities-towns/citiesx020.shp")
shapefile['NAME'] = shapefile['NAME'].str.replace("Saint", 'St.')
shapefile['for_tiger'] = shapefile['STATE_FIPS'] + shapefile['FIPS55']



In [139]:
shapefile.head(3)

,AREA,PERIMETER,CITIESX020,FEATURE,NAME,FIPS,FIPS55,POP,STATE_FIPS,STATE,COUNTY,geometry,for_tiger
0,0.0,0.0,1,"Population 10,000 to 49,999",Aberdeen,53027,00100,16565,53,WA,Grays Harbor County,POINT (-123.81814 46.97494),5300100
1,0.0,0.0,2,"Population less than 10,000",Acme,53073,00275,0,53,WA,Whatcom County,POINT (-122.20778 48.71663),5300275
2,0.0,0.0,3,"Population less than 10,000",Addy,53065,00380,0,53,WA,Stevens County,POINT (-117.83915 48.35600),5300380


In [140]:
ref_arr_cities_with_ceilings = pd.merge(ref_arr_cities_with_ceilings, state_codes, how = 'left', left_on=["state"], right_on=['stname'])

In [141]:
ref_arr_cities_with_ceilings.head(3)

,state,year,statetotal,origin,stateorigin,city,cityorigin,annualtotal,Region of Origin,Country of Origin,FY,Region Refugee Admissions Ceiling,ref_num,stname,st,stusps
0,Alabama,2009,187,Afghanistan,0,Mobile,0,74654,Near East/South Asia,Afghanistan,2009.0,39500.0,349.0,Alabama,01,AL
1,Alabama,2010,159,Afghanistan,13,Mobile,13,73309,Near East/South Asia,Afghanistan,2010.0,38000.0,515.0,Alabama,01,AL
2,Alabama,2011,89,Afghanistan,5,Mobile,5,56424,Near East/South Asia,Afghanistan,2011.0,35500.0,428.0,Alabama,01,AL


In [142]:
ref_arr_cities_with_ceilings[' stusps'] = ref_arr_cities_with_ceilings[' stusps'].str.strip()

In [143]:
all_cities = pd.merge(ref_arr_cities_with_ceilings, shapefile, how = 'left', left_on=['city', ' stusps'], right_on=['NAME', 'STATE'])
all_cities.head()

,state,year,statetotal,origin,stateorigin,city,cityorigin,annualtotal,Region of Origin,Country of Origin,...,FEATURE,NAME,FIPS,FIPS55,POP,STATE_FIPS,STATE,COUNTY,geometry,for_tiger
0,Alabama,2009,187,Afghanistan,0,Mobile,0,74654,Near East/South Asia,Afghanistan,...,"Population 100,000 to 249,999 County Seat",Mobile,01097,50000,196278.0,01,AL,Mobile County,POINT (-88.04310 30.69476),0150000
1,Alabama,2010,159,Afghanistan,13,Mobile,13,73309,Near East/South Asia,Afghanistan,...,"Population 100,000 to 249,999 County Seat",Mobile,01097,50000,196278.0,01,AL,Mobile County,POINT (-88.04310 30.69476),0150000
2,Alabama,2011,89,Afghanistan,5,Mobile,5,56424,Near East/South Asia,Afghanistan,...,"Population 100,000 to 249,999 County Seat",Mobile,01097,50000,196278.0,01,AL,Mobile County,POINT (-88.04310 30.69476),0150000
3,Alabama,2012,145,Afghanistan,7,Mobile,7,58238,Near East/South Asia,Afghanistan,...,"Population 100,000 to 249,999 County Seat",Mobile,01097,50000,196278.0,01,AL,Mobile County,POINT (-88.04310 30.69476),0150000
4,Alabama,2013,129,Afghanistan,9,Mobile,9,69926,Near East/South Asia,Afghanistan,...,"Population 100,000 to 249,999 County Seat",Mobile,01097,50000,196278.0,01,AL,Mobile County,POINT (-88.04310 30.69476),0150000


In [144]:
foreign_born_full.head(3)

,NAME,GEO_ID,Variable,value,Variable_Type,country,FY,for_tiger,GEO_Type,State_Code,stname,st,stusps,NAME_First
0,United States,0100000US,B05006_001E,44525458.0,Estimate,Total,2017,,Country,,NaN,NaN,NaN,United States
1,South Region,0200000US3,B05006_001E,14900362.0,Estimate,Total,2017,3,Region,3,NaN,NaN,NaN,South Region
2,West Region,0200000US4,B05006_001E,15127240.0,Estimate,Total,2017,4,Region,4,NaN,NaN,NaN,West Region


In [145]:
all_cities_1 = pd.merge(all_cities, foreign_born_full[['NAME','GEO_ID','Variable','value','Variable_Type','country','FY','for_tiger','State_Code', 'stname', 'NAME_First']], how = 'left', left_on=["for_tiger", "origin","year"], right_on=['for_tiger',"country","FY"])

In [146]:
all_cities_1 = all_cities_1[(all_cities_1['year'] == "2016") | (all_cities_1['year'] == "2017")  | (all_cities_1['year'] == "2018")]
all_cities_1.head(3)

,state,year,statetotal,origin,stateorigin,city,cityorigin,annualtotal,Region of Origin,Country of Origin,...,NAME_y,GEO_ID,Variable,value,Variable_Type,country,FY_y,State_Code,stname_y,NAME_First
7,Alabama,2016,120,Afghanistan,9,Mobile,9,84994,Near East/South Asia,Afghanistan,...,"Mobile city, Alabama",1600000US0150000,B05006_057E,0.0,Estimate,Afghanistan,2016,01,Alabama,Mobile
8,Alabama,2017,63,Afghanistan,9,Mobile,9,53716,Near East/South Asia,Afghanistan,...,"Mobile city, Alabama",1600000US0150000,B05006_057E,0.0,Estimate,Afghanistan,2017,01,Alabama,Mobile
9,Alabama,2018,38,Afghanistan,0,Mobile,0,22491,Near East/South Asia,Afghanistan,...,"Mobile city, Alabama",1600000US0150000,B05006_057E,0.0,Estimate,Afghanistan,2018,01,Alabama,Mobile


Then on names to check

In [147]:
all_cities_2 = pd.merge(all_cities_1, foreign_born_full, how = 'left', left_on=['city', 'state', 'origin', "year"], right_on=['NAME_First', 'stname' , 'country', 'FY'])

In [148]:
all_cities_2 = all_cities_2[(all_cities_2['year'] == "2016") | (all_cities_2['year'] == "2017")  | (all_cities_2['year'] == "2018")]
all_cities_2.head(3)

,state,year,statetotal,origin,stateorigin,city,cityorigin,annualtotal,Region of Origin,Country of Origin,...,Variable_Type_y,country_y,FY,for_tiger_y,GEO_Type,State_Code_y,stname,st_y,stusps_y,NAME_First_y
0,Alabama,2016,120,Afghanistan,9,Mobile,9,84994,Near East/South Asia,Afghanistan,...,Estimate,Afghanistan,2016,0150000,City,01,Alabama,01,AL,Mobile
1,Alabama,2017,63,Afghanistan,9,Mobile,9,53716,Near East/South Asia,Afghanistan,...,Estimate,Afghanistan,2017,0150000,City,01,Alabama,01,AL,Mobile
2,Alabama,2018,38,Afghanistan,0,Mobile,0,22491,Near East/South Asia,Afghanistan,...,Estimate,Afghanistan,2018,0150000,City,01,Alabama,01,AL,Mobile


In [149]:
all_cities_2['NAME_y'] = all_cities_2['NAME_y'].astype('str')

In [150]:
all_cities_2['NAME_y'] = all_cities_2.apply(lambda x : x['NAME_y'] if (x['NAME_y'] != np.nan)  else x['NAME'], axis=1)

all_cities_2['GEO_ID_x'] = all_cities_2.apply(lambda x : x['GEO_ID_x'] if (x['GEO_ID_x'] != np.nan)  else x['GEO_ID_y'], axis=1)

all_cities_2['Variable_x'] = all_cities_2.apply(lambda x : x['Variable_x'] if (x['Variable_x'] != np.nan)  else x['Variable_y'], axis=1)

all_cities_2['value_x'] = all_cities_2.apply(lambda x : x['value_x'] if (x['value_x'] != np.nan)  else x['value_y'], axis=1)

all_cities_2['Variable_Type_x'] = all_cities_2.apply(lambda x : x['Variable_Type_x'] if (x['Variable_Type_x'] != np.nan)  else x['Variable_Type_y'], axis=1)

# Delete Country, Delete FY, Delete State_Code, stname_Y, and NAME_first

In [151]:
all_cities_2 = all_cities_2.iloc[:,0:34]

In [152]:
all_cities_2 = all_cities_2.drop_duplicates()

Then resolve

In [153]:
all_cities = all_cities_2
all_cities[all_cities['Variable_Type_x'].isna()].shape
# 12,313 went unmatched this way. 

(18478, 34)

In [154]:
all_cities.columns

Index(['state', 'year', 'statetotal', 'origin', 'stateorigin', 'city',
       'cityorigin', 'annualtotal', 'Region of Origin', 'Country of Origin',
       'FY_x', 'Region Refugee Admissions Ceiling', 'ref_num', 'stname_x',
       ' st_x', ' stusps_x', 'AREA', 'PERIMETER', 'CITIESX020', 'FEATURE',
       'NAME_x', 'FIPS', 'FIPS55', 'POP', 'STATE_FIPS', 'STATE', 'COUNTY',
       'geometry', 'for_tiger_x', 'NAME_y', 'GEO_ID_x', 'Variable_x',
       'value_x', 'Variable_Type_x'],
      dtype='object')

So the first row, for example, says in 2016, there were 9 refugees from Afghanistan that went to Mobile out of the 9 Afghans that went to Alabama in 2016. 120 Refugees overall went to Alabama in 2016. 84,994 total refugees came in 2016 to the US. The refugee admissions ceiling for the Near East/South Asia region in 2016 was 38,000 refugees. 2,737 Afghani refugees came into the US in 2016. According to the census, there were 0 others born in Afghanistan in Mobile in 2016. 

In [155]:
all_cities = all_cities.rename(columns={"FY_x": "FY", "stname_x": "stname", " st_x": "st", "stusps_x": "stusps", "NAME_x": "Name", "for_tiger_x": "for_tiger", "NAME_y": "Census_Name", "GEO_ID_x": "GEO_ID", "Variable_x": "Variable", "value_x": "value", "Variable_Type_x": "Variable_Type"})
all_cities.columns

Index(['state', 'year', 'statetotal', 'origin', 'stateorigin', 'city',
       'cityorigin', 'annualtotal', 'Region of Origin', 'Country of Origin',
       'FY', 'Region Refugee Admissions Ceiling', 'ref_num', 'stname', 'st',
       ' stusps_x', 'AREA', 'PERIMETER', 'CITIESX020', 'FEATURE', 'Name',
       'FIPS', 'FIPS55', 'POP', 'STATE_FIPS', 'STATE', 'COUNTY', 'geometry',
       'for_tiger', 'Census_Name', 'GEO_ID', 'Variable', 'value',
       'Variable_Type'],
      dtype='object')

In [156]:
all_cities['year'].value_counts()

2016    9929
2017    9914
2018    9900
Name: year, dtype: int64

In [157]:
#all_cities.to_csv('all_cities_originpops.csv', index=False, encoding = "utf_8_sig")

In [158]:
check_na = all_cities[all_cities['value'].isna()]
check_na.shape

(18478, 34)

In [159]:
check_notna = all_cities[all_cities['value'].notna()]
check_notna.shape

(11265, 34)

In [160]:
check_notna[check_notna['value'] > 0].head()

,state,year,statetotal,origin,stateorigin,city,cityorigin,annualtotal,Region of Origin,Country of Origin,...,STATE_FIPS,STATE,COUNTY,geometry,for_tiger,Census_Name,GEO_ID,Variable,value,Variable_Type
222,Arizona,2016,4110,Afghanistan,167,Phoenix,112,84994,Near East/South Asia,Afghanistan,...,04,AZ,Maricopa County,POINT (-112.07527 33.44825),0455000,"Phoenix city, Arizona",1600000US0455000,B05006_057E,1500.0,Estimate
264,Arizona,2016,4110,Burma,264,Phoenix,137,84994,East Asia,Burma,...,04,AZ,Maricopa County,POINT (-112.07527 33.44825),0455000,"Phoenix city, Arizona",1600000US0455000,B05006_068E,410.0,Estimate
288,Arizona,2016,4110,Cameroon,5,Phoenix,4,84994,Africa,Cameroon,...,04,AZ,Maricopa County,POINT (-112.07527 33.44825),0455000,"Phoenix city, Arizona",1600000US0455000,B05006_101E,2123.0,Estimate
312,Arizona,2016,4110,China,1,Phoenix,0,84994,East Asia,China,...,04,AZ,Maricopa County,POINT (-112.07527 33.44825),0455000,"Phoenix city, Arizona",1600000US0455000,B05006_049E,5287.0,Estimate
318,Arizona,2016,4110,Colombia,7,Phoenix,0,84994,Latin America/Caribbean,Colombia,...,04,AZ,Maricopa County,POINT (-112.07527 33.44825),0455000,"Phoenix city, Arizona",1600000US0455000,B05006_159E,51.0,Estimate


So the first row, for example, says in 2016, there were 112 refugees from Afghanistan that went to Phoenix out of the 167 Afghans that went to Arizona in 2016. 4,110 Refugees overall went to Arizona in 2016. 84,994 total refugees came in 2016 to the US. The refugee admissions ceiling for the Near East/South Asia region in 2016 was 38,000 refugees. 2,737 Afghani refugees came into the US in 2016. According to the census, there were 1,500 others born in Afghanistan in Phoenix in 2016. 

In [161]:
tract_zip = pd.read_excel(import_bucket + "TRACT_ZIP_092021.xlsx", sheet_name = "TRACT_ZIP_092021", skiprows = 0)
tract_zip.head()

,TRACT,ZIP,USPS_ZIP_PREF_CITY,USPS_ZIP_PREF_STATE,RES_RATIO,BUS_RATIO,OTH_RATIO,TOT_RATIO
0,1001020100,36067,PRATTVILLE,AL,1.000000,1.000000,1.00,1.000000
1,1001020200,36067,PRATTVILLE,AL,1.000000,1.000000,1.00,1.000000
2,1001020300,36067,PRATTVILLE,AL,1.000000,1.000000,1.00,1.000000
3,1001020400,36067,PRATTVILLE,AL,0.024142,0.583851,0.25,0.070064
4,1001020400,36066,PRATTVILLE,AL,0.975858,0.416149,0.75,0.929936


4. Food Data

In [162]:
grocery_stores.head()

,CensusTract,State,County,LILATracts_1And10,LILATracts_halfAnd10,LILATracts_1And20,LILATracts_Vehicle,Urban,Rural,LA1and10,...,lapop20,lapop20share,lalowi20,lalowi20share,lakids20,lakids20share,laseniors20,laseniors20share,lahunv20,lahunv20share
0,1001020100,AL,Autauga,0,0,0,0,1,0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1001020200,AL,Autauga,0,0,0,0,1,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1001020300,AL,Autauga,0,0,0,0,1,0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1001020400,AL,Autauga,0,0,0,0,1,0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1001020500,AL,Autauga,0,0,0,0,1,0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [163]:
a = grocery_stores.columns.tolist()
cols = pd.Series( (v for v in a) )
cols.to_csv('grocery_code_book.csv', index=False, encoding = "utf_8_sig")

In [164]:
grocery_stores_cross = pd.merge(grocery_stores, tract_zip, how = 'left', left_on=['CensusTract'], right_on=['TRACT'])
grocery_stores_cross.head()


,CensusTract,State,County,LILATracts_1And10,LILATracts_halfAnd10,LILATracts_1And20,LILATracts_Vehicle,Urban,Rural,LA1and10,...,lahunv20,lahunv20share,TRACT,ZIP,USPS_ZIP_PREF_CITY,USPS_ZIP_PREF_STATE,RES_RATIO,BUS_RATIO,OTH_RATIO,TOT_RATIO
0,1001020100,AL,Autauga,0,0,0,0,1,0,1,...,0.0,0.0,1.001020e+09,36067.0,PRATTVILLE,AL,1.000000,1.000000,1.00,1.000000
1,1001020200,AL,Autauga,0,0,0,0,1,0,0,...,0.0,0.0,1.001020e+09,36067.0,PRATTVILLE,AL,1.000000,1.000000,1.00,1.000000
2,1001020300,AL,Autauga,0,0,0,0,1,0,1,...,0.0,0.0,1.001020e+09,36067.0,PRATTVILLE,AL,1.000000,1.000000,1.00,1.000000
3,1001020400,AL,Autauga,0,0,0,0,1,0,1,...,0.0,0.0,1.001020e+09,36067.0,PRATTVILLE,AL,0.024142,0.583851,0.25,0.070064
4,1001020400,AL,Autauga,0,0,0,0,1,0,1,...,0.0,0.0,1.001020e+09,36066.0,PRATTVILLE,AL,0.975858,0.416149,0.75,0.929936


Tracts don't line up perfectly. 

In [165]:
grocery_stores_cross = grocery_stores_cross.drop_duplicates(subset=['CensusTract', 'USPS_ZIP_PREF_CITY'], keep='first')

In [166]:
sum_cols = ["LA1and10", "LA1and20", "LAhalfand10", "lahunv1", "lahunv10", "lahunv10share", "lahunv20", "lahunvhalf", "lakids1", "lakids10", "lakids20", "lakidshalf", "lalowi1", "lalowi10", "lalowi20", "lalowihalf", "lapop1", "lapop10", "lapop20", "lapophalf", "laseniors1", "laseniors10", "laseniors20", "laseniorshalf", "LATracts_half", "LATracts1", "LATracts10", "LATracts20", "LILATracts_1And10", "LILATracts_1And20", "LILATracts_halfAnd10", "LILATracts_Vehicle", "LowIncomeTracts", "NUMGQTRS", "OHU2010", "POP2010", "Rural"]

In [167]:
perc_cols = ["lahunv1share", "lahunv20share", "lahunvhalfshare", "lakids10share", "lakids1share", "lakids20share", "lakidshalfshare", "lalowi10share", "lalowi1share", "lalowi20share", "lalowihalfshare", "lapop10share", "lapop1share", "lapop20share", "lapophalfshare", "laseniors10share", "laseniors1share", "laseniors20share", "laseniorshalfshare", "PCTGQTRS"]

In [168]:
grocery_stores_cross_city_sum = grocery_stores_cross.groupby(by=["USPS_ZIP_PREF_CITY", "USPS_ZIP_PREF_STATE"], dropna=False, as_index=False)[sum_cols].sum()
grocery_stores_cross_city_perc = grocery_stores_cross.groupby(by=["USPS_ZIP_PREF_CITY", "USPS_ZIP_PREF_STATE"], dropna=False, as_index=False)[perc_cols].mean()
grocery_stores_cross_city = pd.merge(grocery_stores_cross_city_sum , grocery_stores_cross_city_perc, how = 'left', left_on=["USPS_ZIP_PREF_CITY", "USPS_ZIP_PREF_STATE"], right_on=["USPS_ZIP_PREF_CITY", "USPS_ZIP_PREF_STATE"])
grocery_stores_cross_city.head()

,USPS_ZIP_PREF_CITY,USPS_ZIP_PREF_STATE,LA1and10,LA1and20,LAhalfand10,lahunv1,lahunv10,lahunv10share,lahunv20,lahunvhalf,...,lalowihalfshare,lapop10share,lapop1share,lapop20share,lapophalfshare,laseniors10share,laseniors1share,laseniors20share,laseniorshalfshare,PCTGQTRS
0,AARONSBURG,PA,0,0,0,209.984633,10.357654,0.005252,0.0,239.248975,...,0.416967,0.029504,0.856552,0.000000,0.980039,0.029926,0.835830,0.000000,0.980914,0.000178
1,ABBEVILLE,AL,0,0,0,252.627763,19.839876,0.026785,0.0,287.400672,...,0.412637,0.049878,0.838765,0.000000,0.911957,0.040895,0.818570,0.000000,0.900947,0.015530
2,ABBEVILLE,GA,0,0,0,296.716769,11.000041,0.015082,0.0,360.166665,...,0.460736,0.022851,0.662139,0.000000,0.894619,0.032414,0.703005,0.000000,0.870594,0.129919
3,ABBEVILLE,LA,2,1,4,337.176274,0.280884,0.000139,0.0,955.641377,...,0.325262,0.052391,0.635451,0.008763,0.852075,0.057508,0.597433,0.015197,0.825015,0.006951
4,ABBEVILLE,MS,3,1,3,131.490960,12.823896,0.007549,0.0,146.969427,...,0.350146,0.177099,0.915544,0.000000,0.989756,0.224193,0.950075,0.000000,0.989178,0.000000


In [169]:
farmers_markets.head()

,Store Name,Street Number,Street Name,Addl Address,City,State,Zip,County Code,County Name
0,Anchor Point Saterday Farmers Market,70961,Sarah Ln,NaN,Anchor Point,AK,99556,122,KENAI PENINSULA
1,Anchorage Farmers Market,1420,Cordova St,NaN,Anchorage,AK,99501,20,ANCHORAGE
2,Black Bear Farms,12501,E Marsh Rd,NaN,Palmer,AK,99645,170,MATANUSKA SUSITNA
3,Cool Cache Farms,47110,Autumn Rd,NaN,Kenai,AK,99611,122,KENAI PENINSULA
4,Dandelion Acres,4412,N Dogwood Rd,NaN,Kenai,AK,99611,122,KENAI PENINSULA


In [170]:
farmers_markets['City'] = farmers_markets['City'].map(lambda x: x.title())
farmers_markets_city = farmers_markets.groupby(by=["City", "State"], dropna=False, as_index=False).agg({'Store Name':'count'})
farmers_markets_city.columns.values[2] = "farm_mark_count"

farmers_markets_city.head()

,City,State,farm_mark_count
0,Abingdon,VA,1
1,Abita Springs,LA,1
2,Absecon,NJ,1
3,Achille,OK,1
4,Acton,MA,1


In [171]:
all_cities.columns

Index(['state', 'year', 'statetotal', 'origin', 'stateorigin', 'city',
       'cityorigin', 'annualtotal', 'Region of Origin', 'Country of Origin',
       'FY', 'Region Refugee Admissions Ceiling', 'ref_num', 'stname', 'st',
       ' stusps_x', 'AREA', 'PERIMETER', 'CITIESX020', 'FEATURE', 'Name',
       'FIPS', 'FIPS55', 'POP', 'STATE_FIPS', 'STATE', 'COUNTY', 'geometry',
       'for_tiger', 'Census_Name', 'GEO_ID', 'Variable', 'value',
       'Variable_Type'],
      dtype='object')

In [172]:
all_cities[' stusps_x'] = all_cities[' stusps_x'].str.strip()

grocery_stores_cross_city["USPS_ZIP_PREF_CITY"] = grocery_stores_cross_city["USPS_ZIP_PREF_CITY"].astype("str")

grocery_stores_cross_city["USPS_ZIP_PREF_CITY"] = grocery_stores_cross_city["USPS_ZIP_PREF_CITY"].map(lambda x: x.title())

all_cities_vouchfood = pd.merge(all_cities, grocery_stores_cross_city, how = 'left', left_on=['city',' stusps_x'], right_on=["USPS_ZIP_PREF_CITY", "USPS_ZIP_PREF_STATE"])
all_cities_vouchfood.head()

,state,year,statetotal,origin,stateorigin,city,cityorigin,annualtotal,Region of Origin,Country of Origin,...,lalowihalfshare,lapop10share,lapop1share,lapop20share,lapophalfshare,laseniors10share,laseniors1share,laseniors20share,laseniorshalfshare,PCTGQTRS
0,Alabama,2016,120,Afghanistan,9,Mobile,9,84994,Near East/South Asia,Afghanistan,...,0.328088,0.000000,0.357608,0.0,0.721217,0.000000,0.342835,0.0,0.698883,0.02913
1,Alabama,2017,63,Afghanistan,9,Mobile,9,53716,Near East/South Asia,Afghanistan,...,0.328088,0.000000,0.357608,0.0,0.721217,0.000000,0.342835,0.0,0.698883,0.02913
2,Alabama,2018,38,Afghanistan,0,Mobile,0,22491,Near East/South Asia,Afghanistan,...,0.328088,0.000000,0.357608,0.0,0.721217,0.000000,0.342835,0.0,0.698883,0.02913
3,Alabama,2016,120,Burma,3,Auburn,0,84994,East Asia,Burma,...,0.418359,0.009169,0.693626,0.0,0.890827,0.011503,0.698465,0.0,0.896511,0.06481
4,Alabama,2017,63,Burma,1,Auburn,0,53716,East Asia,Burma,...,0.418359,0.009169,0.693626,0.0,0.890827,0.011503,0.698465,0.0,0.896511,0.06481


In [173]:
all_cities_vouchfood = pd.merge(all_cities_vouchfood, farmers_markets_city, how = 'left', left_on=["USPS_ZIP_PREF_CITY", "USPS_ZIP_PREF_STATE"], right_on=["City", "State"])
all_cities_vouchfood.head()

,state,year,statetotal,origin,stateorigin,city,cityorigin,annualtotal,Region of Origin,Country of Origin,...,lapop20share,lapophalfshare,laseniors10share,laseniors1share,laseniors20share,laseniorshalfshare,PCTGQTRS,City,State,farm_mark_count
0,Alabama,2016,120,Afghanistan,9,Mobile,9,84994,Near East/South Asia,Afghanistan,...,0.0,0.721217,0.000000,0.342835,0.0,0.698883,0.02913,Mobile,AL,2.0
1,Alabama,2017,63,Afghanistan,9,Mobile,9,53716,Near East/South Asia,Afghanistan,...,0.0,0.721217,0.000000,0.342835,0.0,0.698883,0.02913,Mobile,AL,2.0
2,Alabama,2018,38,Afghanistan,0,Mobile,0,22491,Near East/South Asia,Afghanistan,...,0.0,0.721217,0.000000,0.342835,0.0,0.698883,0.02913,Mobile,AL,2.0
3,Alabama,2016,120,Burma,3,Auburn,0,84994,East Asia,Burma,...,0.0,0.890827,0.011503,0.698465,0.0,0.896511,0.06481,NaN,NaN,NaN
4,Alabama,2017,63,Burma,1,Auburn,0,53716,East Asia,Burma,...,0.0,0.890827,0.011503,0.698465,0.0,0.896511,0.06481,NaN,NaN,NaN


In [174]:
all_cities_vouchfood.columns

Index(['state', 'year', 'statetotal', 'origin', 'stateorigin', 'city',
       'cityorigin', 'annualtotal', 'Region of Origin', 'Country of Origin',
       'FY', 'Region Refugee Admissions Ceiling', 'ref_num', 'stname', 'st',
       ' stusps_x', 'AREA', 'PERIMETER', 'CITIESX020', 'FEATURE', 'Name',
       'FIPS', 'FIPS55', 'POP', 'STATE_FIPS', 'STATE', 'COUNTY', 'geometry',
       'for_tiger', 'Census_Name', 'GEO_ID', 'Variable', 'value',
       'Variable_Type', 'USPS_ZIP_PREF_CITY', 'USPS_ZIP_PREF_STATE',
       'LA1and10', 'LA1and20', 'LAhalfand10', 'lahunv1', 'lahunv10',
       'lahunv10share', 'lahunv20', 'lahunvhalf', 'lakids1', 'lakids10',
       'lakids20', 'lakidshalf', 'lalowi1', 'lalowi10', 'lalowi20',
       'lalowihalf', 'lapop1', 'lapop10', 'lapop20', 'lapophalf', 'laseniors1',
       'laseniors10', 'laseniors20', 'laseniorshalf', 'LATracts_half',
       'LATracts1', 'LATracts10', 'LATracts20', 'LILATracts_1And10',
       'LILATracts_1And20', 'LILATracts_halfAnd10', 'LILATr

In [175]:
all_cities_vouchfood = all_cities_vouchfood.drop(['Country of Origin', 'FY', 'stname', 'Name', 'USPS_ZIP_PREF_CITY', 'USPS_ZIP_PREF_STATE', 'City', 'State'], axis=1)
all_cities_vouchfood.head()

,state,year,statetotal,origin,stateorigin,city,cityorigin,annualtotal,Region of Origin,Region Refugee Admissions Ceiling,...,lapop10share,lapop1share,lapop20share,lapophalfshare,laseniors10share,laseniors1share,laseniors20share,laseniorshalfshare,PCTGQTRS,farm_mark_count
0,Alabama,2016,120,Afghanistan,9,Mobile,9,84994,Near East/South Asia,38000.0,...,0.000000,0.357608,0.0,0.721217,0.000000,0.342835,0.0,0.698883,0.02913,2.0
1,Alabama,2017,63,Afghanistan,9,Mobile,9,53716,Near East/South Asia,40000.0,...,0.000000,0.357608,0.0,0.721217,0.000000,0.342835,0.0,0.698883,0.02913,2.0
2,Alabama,2018,38,Afghanistan,0,Mobile,0,22491,Near East/South Asia,17500.0,...,0.000000,0.357608,0.0,0.721217,0.000000,0.342835,0.0,0.698883,0.02913,2.0
3,Alabama,2016,120,Burma,3,Auburn,0,84994,East Asia,14000.0,...,0.009169,0.693626,0.0,0.890827,0.011503,0.698465,0.0,0.896511,0.06481,NaN
4,Alabama,2017,63,Burma,1,Auburn,0,53716,East Asia,12000.0,...,0.009169,0.693626,0.0,0.890827,0.011503,0.698465,0.0,0.896511,0.06481,NaN


5. Affiliate Data

In [176]:
affiliate_full.head()

,City,State,FY,CWS,ECDC,EMM,HIAS,IRC,LIRS,USCCB,USCRI,WR
0,Abilene,TX,2021,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,Akron,OH,2021,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,Albany,NY,2021,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,Albuquerque,NM,2021,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,Allentown,PA,2021,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [177]:
affiliate_full['State'] = affiliate_full['State'].str.strip()

In [178]:
affiliate_full['FY'] = affiliate_full['FY'].astype("str")

In [179]:
all_cities_vouchfoodaff = pd.merge(all_cities_vouchfood, affiliate_full, how = 'left', left_on=["city"," stusps_x","year"], right_on=["City","State","FY"])
all_cities_vouchfoodaff.head()

,state,year,statetotal,origin,stateorigin,city,cityorigin,annualtotal,Region of Origin,Region Refugee Admissions Ceiling,...,FY,CWS,ECDC,EMM,HIAS,IRC,LIRS,USCCB,USCRI,WR
0,Alabama,2016,120,Afghanistan,9,Mobile,9,84994,Near East/South Asia,38000.0,...,2016,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,Alabama,2017,63,Afghanistan,9,Mobile,9,53716,Near East/South Asia,40000.0,...,2017,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,Alabama,2018,38,Afghanistan,0,Mobile,0,22491,Near East/South Asia,17500.0,...,2018,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,Alabama,2016,120,Burma,3,Auburn,0,84994,East Asia,14000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Alabama,2017,63,Burma,1,Auburn,0,53716,East Asia,12000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [180]:
#all_cities_vouchfoodaff.to_csv('all_cities_vouchfoodaff.csv', index=False, encoding = "utf_8_sig")

In [181]:
all_cities_vouchfoodaff_2018 = all_cities_vouchfoodaff[all_cities_vouchfoodaff['year'] == "2018"]
all_cities_vouchfoodaff_2018.head()

,state,year,statetotal,origin,stateorigin,city,cityorigin,annualtotal,Region of Origin,Region Refugee Admissions Ceiling,...,FY,CWS,ECDC,EMM,HIAS,IRC,LIRS,USCCB,USCRI,WR
2,Alabama,2018,38,Afghanistan,0,Mobile,0,22491,Near East/South Asia,17500.0,...,2018,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
5,Alabama,2018,38,Burma,1,Auburn,0,22491,East Asia,5000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,Alabama,2018,38,Burma,1,Birmingham,1,22491,East Asia,5000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11,Alabama,2018,38,Burma,1,Dothan,0,22491,East Asia,5000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14,Alabama,2018,38,Burma,1,Mobile,0,22491,East Asia,5000.0,...,2018,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [182]:
all_cities_vouchfoodaff_2018.to_csv('all_cities_vouchfoodaff_2018.csv', index=False, encoding = "utf_8_sig")

In [1482]:
#all_cities_vouchfood.to_csv('all_cities_vouchfood.csv', index=False, encoding = "utf_8_sig")

In [183]:
all_cities_vouchfood_2018 = all_cities_vouchfood[all_cities_vouchfood['year'] == "2018"]
all_cities_vouchfood_2018.head()

,state,year,statetotal,origin,stateorigin,city,cityorigin,annualtotal,Region of Origin,Region Refugee Admissions Ceiling,...,lapop10share,lapop1share,lapop20share,lapophalfshare,laseniors10share,laseniors1share,laseniors20share,laseniorshalfshare,PCTGQTRS,farm_mark_count
2,Alabama,2018,38,Afghanistan,0,Mobile,0,22491,Near East/South Asia,17500.0,...,0.000000,0.357608,0.0,0.721217,0.000000,0.342835,0.0,0.698883,0.029130,2.0
5,Alabama,2018,38,Burma,1,Auburn,0,22491,East Asia,5000.0,...,0.009169,0.693626,0.0,0.890827,0.011503,0.698465,0.0,0.896511,0.064810,NaN
8,Alabama,2018,38,Burma,1,Birmingham,1,22491,East Asia,5000.0,...,0.000046,0.431261,0.0,0.789475,0.000044,0.426281,0.0,0.791517,0.022144,6.0
11,Alabama,2018,38,Burma,1,Dothan,0,22491,East Asia,5000.0,...,0.000000,0.494310,0.0,0.763160,0.000000,0.485990,0.0,0.767824,0.017158,NaN
14,Alabama,2018,38,Burma,1,Mobile,0,22491,East Asia,5000.0,...,0.000000,0.357608,0.0,0.721217,0.000000,0.342835,0.0,0.698883,0.029130,2.0


In [1484]:
#all_cities_vouchfood_2018.to_csv('all_cities_vouchfood_2018.csv', index=False, encoding = "utf_8_sig")

5. Merge with city shapefiles

In [184]:
all_cities_vouchfood_geo = all_cities_vouchfood

In [185]:
all_cities_vouchfood_geo = all_cities_vouchfood_geo[all_cities_vouchfood_geo['geometry'].notna()]

In [186]:
all_cities_vouchfood_geo['coords'] = all_cities_vouchfood_geo['geometry'].astype('str')

In [187]:
def get_longitude(x):
    y = x.split(" ")[1].replace("(", "")
    return y
def get_latitude(x):
    y = x.split(" ")[2].replace(")", "")
    return y

In [188]:
all_cities_vouchfood_geo['latitude'] = all_cities_vouchfood_geo['coords'].map(lambda x: get_latitude(x))
all_cities_vouchfood_geo['longitude'] = all_cities_vouchfood_geo['coords'].map(lambda x: get_longitude(x))

In [189]:
all_cities_vouchfood_geo.head()

,state,year,statetotal,origin,stateorigin,city,cityorigin,annualtotal,Region of Origin,Region Refugee Admissions Ceiling,...,lapophalfshare,laseniors10share,laseniors1share,laseniors20share,laseniorshalfshare,PCTGQTRS,farm_mark_count,coords,latitude,longitude
0,Alabama,2016,120,Afghanistan,9,Mobile,9,84994,Near East/South Asia,38000.0,...,0.721217,0.000000,0.342835,0.0,0.698883,0.02913,2.0,POINT (-88.04309844970703 30.69476127624512),30.69476127624512,-88.04309844970703
1,Alabama,2017,63,Afghanistan,9,Mobile,9,53716,Near East/South Asia,40000.0,...,0.721217,0.000000,0.342835,0.0,0.698883,0.02913,2.0,POINT (-88.04309844970703 30.69476127624512),30.69476127624512,-88.04309844970703
2,Alabama,2018,38,Afghanistan,0,Mobile,0,22491,Near East/South Asia,17500.0,...,0.721217,0.000000,0.342835,0.0,0.698883,0.02913,2.0,POINT (-88.04309844970703 30.69476127624512),30.69476127624512,-88.04309844970703
3,Alabama,2016,120,Burma,3,Auburn,0,84994,East Asia,14000.0,...,0.890827,0.011503,0.698465,0.0,0.896511,0.06481,NaN,POINT (-85.48087310791016 32.61021423339844),32.61021423339844,-85.48087310791016
4,Alabama,2017,63,Burma,1,Auburn,0,53716,East Asia,12000.0,...,0.890827,0.011503,0.698465,0.0,0.896511,0.06481,NaN,POINT (-85.48087310791016 32.61021423339844),32.61021423339844,-85.48087310791016


In [190]:
all_cities_vouchfood_geo.to_csv('all_cities_vouchfood_geo.csv', index=False, encoding = "utf_8_sig")

In [191]:
all_cities_vouchfood_geo_2018 = all_cities_vouchfood_geo[all_cities_vouchfood_geo['year'] == "2018"]
all_cities_vouchfood_geo_2018.head()

,state,year,statetotal,origin,stateorigin,city,cityorigin,annualtotal,Region of Origin,Region Refugee Admissions Ceiling,...,lapophalfshare,laseniors10share,laseniors1share,laseniors20share,laseniorshalfshare,PCTGQTRS,farm_mark_count,coords,latitude,longitude
2,Alabama,2018,38,Afghanistan,0,Mobile,0,22491,Near East/South Asia,17500.0,...,0.721217,0.000000,0.342835,0.0,0.698883,0.029130,2.0,POINT (-88.04309844970703 30.69476127624512),30.69476127624512,-88.04309844970703
5,Alabama,2018,38,Burma,1,Auburn,0,22491,East Asia,5000.0,...,0.890827,0.011503,0.698465,0.0,0.896511,0.064810,NaN,POINT (-85.48087310791016 32.61021423339844),32.61021423339844,-85.48087310791016
8,Alabama,2018,38,Burma,1,Birmingham,1,22491,East Asia,5000.0,...,0.789475,0.000044,0.426281,0.0,0.791517,0.022144,6.0,POINT (-86.80226898193359 33.52098846435547),33.52098846435547,-86.80226898193359
11,Alabama,2018,38,Burma,1,Dothan,0,22491,East Asia,5000.0,...,0.763160,0.000000,0.485990,0.0,0.767824,0.017158,NaN,POINT (-85.39019775390625 31.22351264953613),31.22351264953613,-85.39019775390625
14,Alabama,2018,38,Burma,1,Mobile,0,22491,East Asia,5000.0,...,0.721217,0.000000,0.342835,0.0,0.698883,0.029130,2.0,POINT (-88.04309844970703 30.69476127624512),30.69476127624512,-88.04309844970703


In [192]:
all_cities_vouchfood_geo_2018.to_csv('all_cities_vouchfood_geo_2018.csv', index=False, encoding = "utf_8_sig")

# For data with aff

In [193]:
all_cities_vouchfoodaff_geo = all_cities_vouchfoodaff

In [194]:
all_cities_vouchfoodaff_geo = all_cities_vouchfoodaff_geo[all_cities_vouchfoodaff_geo['geometry'].notna()]

In [195]:
all_cities_vouchfoodaff_geo['coords'] = all_cities_vouchfoodaff_geo['geometry'].astype('str')

In [196]:
all_cities_vouchfoodaff_geo['latitude'] = all_cities_vouchfoodaff_geo['coords'].map(lambda x: get_latitude(x))
all_cities_vouchfoodaff_geo['longitude'] = all_cities_vouchfoodaff_geo['coords'].map(lambda x: get_longitude(x))

In [197]:
all_cities_vouchfoodaff_geo.head()

,state,year,statetotal,origin,stateorigin,city,cityorigin,annualtotal,Region of Origin,Region Refugee Admissions Ceiling,...,EMM,HIAS,IRC,LIRS,USCCB,USCRI,WR,coords,latitude,longitude
0,Alabama,2016,120,Afghanistan,9,Mobile,9,84994,Near East/South Asia,38000.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,POINT (-88.04309844970703 30.69476127624512),30.69476127624512,-88.04309844970703
1,Alabama,2017,63,Afghanistan,9,Mobile,9,53716,Near East/South Asia,40000.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,POINT (-88.04309844970703 30.69476127624512),30.69476127624512,-88.04309844970703
2,Alabama,2018,38,Afghanistan,0,Mobile,0,22491,Near East/South Asia,17500.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,POINT (-88.04309844970703 30.69476127624512),30.69476127624512,-88.04309844970703
3,Alabama,2016,120,Burma,3,Auburn,0,84994,East Asia,14000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,POINT (-85.48087310791016 32.61021423339844),32.61021423339844,-85.48087310791016
4,Alabama,2017,63,Burma,1,Auburn,0,53716,East Asia,12000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,POINT (-85.48087310791016 32.61021423339844),32.61021423339844,-85.48087310791016


In [198]:
#all_cities_vouchfoodaff_geo.to_csv('all_cities_vouchfoodaff_geo.csv', index=False, encoding = "utf_8_sig")

In [199]:
all_cities_vouchfoodaff_geo_2018 = all_cities_vouchfoodaff_geo[all_cities_vouchfoodaff_geo['year'] == "2018"]
all_cities_vouchfoodaff_geo_2018.head()

,state,year,statetotal,origin,stateorigin,city,cityorigin,annualtotal,Region of Origin,Region Refugee Admissions Ceiling,...,EMM,HIAS,IRC,LIRS,USCCB,USCRI,WR,coords,latitude,longitude
2,Alabama,2018,38,Afghanistan,0,Mobile,0,22491,Near East/South Asia,17500.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,POINT (-88.04309844970703 30.69476127624512),30.69476127624512,-88.04309844970703
5,Alabama,2018,38,Burma,1,Auburn,0,22491,East Asia,5000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,POINT (-85.48087310791016 32.61021423339844),32.61021423339844,-85.48087310791016
8,Alabama,2018,38,Burma,1,Birmingham,1,22491,East Asia,5000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,POINT (-86.80226898193359 33.52098846435547),33.52098846435547,-86.80226898193359
11,Alabama,2018,38,Burma,1,Dothan,0,22491,East Asia,5000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,POINT (-85.39019775390625 31.22351264953613),31.22351264953613,-85.39019775390625
14,Alabama,2018,38,Burma,1,Mobile,0,22491,East Asia,5000.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,POINT (-88.04309844970703 30.69476127624512),30.69476127624512,-88.04309844970703


In [200]:
#all_cities_vouchfoodaff_geo_2018.to_csv('all_cities_vouchfoodaff_geo_2018.csv', index=False, encoding = "utf_8")

## Census Data Round 2

In [601]:
keyfile='census_key.txt'
#Read api key in from file
with open(keyfile) as key:
    api_key=key.read().strip()
    

In [602]:
#Set variables
year='2018'
dsource='acs'
dname='acs1'
series = "subject"
cols='NAME,group(S0501)'
keyfile='census_key.txt'
geography="combined%20statistical%20area"

#metropolitan%20division:14454&in=metropolitan%20statistical%20area/micropolitan%20statistical%20area:14460&key=YOUR_KEY_GOES_HERE
#consolidated%20city:47500&in=state:09&key=YOUR_KEY_GOES_HERE

In [603]:
def retreive_census(year, dsource,dname,cols,series,keyfile):
    base_url = f'https://api.census.gov/data/{year}/{dsource}/{dname}/{series}'
    #Retrieve data, write output when finished
    data_url = f'{base_url}?get={cols}&for={geography}:*&key={api_key}'
    response=requests.get(data_url)
    outdata=response.json()
    with open(outfile, 'w') as f:
        json.dump(outdata, f)
    print('Data dumped to json file')
    print('File has', len(outdata), 'records')
    #Load data from output file, convert to dataframe, perform some basic operations
    with open(outfile, 'r') as f:
        data=json.load(f)
    df=pd.DataFrame(data[1:], columns=data[0])
    return df

In [604]:
fornav_char = retreive_census(year, dsource,dname,cols,series,keyfile)
fornav_char.head(3)

Data dumped to json file
File has 172 records


,NAME,GEO_ID,NAME,S0501_C01_001E,S0501_C01_001EA,S0501_C01_001M,S0501_C01_001MA,S0501_C01_002E,S0501_C01_002EA,S0501_C01_002M,...,S0501_C05_135MA,S0501_C05_136E,S0501_C05_136EA,S0501_C05_136M,S0501_C05_136MA,S0501_C05_137E,S0501_C05_137EA,S0501_C05_137M,S0501_C05_137MA,combined statistical area
0,"Sacramento-Roseville, CA CSA",330M400US472,"Sacramento-Roseville, CA CSA",2619754,None,-555555555,*****,49.0,None,0.1,...,None,44.2,None,4.8,None,55.8,None,4.8,None,472
1,"Saginaw-Midland-Bay City, MI CSA",330M400US474,"Saginaw-Midland-Bay City, MI CSA",None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,474
2,"St. Louis-St. Charles-Farmington, MO-IL CSA",330M400US476,"St. Louis-St. Charles-Farmington, MO-IL CSA",2909036,None,2197,None,48.6,None,0.1,...,None,67.7,None,7.0,None,32.3,None,7.0,None,476


In [608]:
fornav_char = fornav_char.loc[:, ~fornav_char.columns.duplicated()]
fornav_char.columns

Index(['NAME', 'GEO_ID', 'S0501_C01_001E', 'S0501_C01_001EA', 'S0501_C01_001M',
       'S0501_C01_001MA', 'S0501_C01_002E', 'S0501_C01_002EA',
       'S0501_C01_002M', 'S0501_C01_002MA',
       ...
       'S0501_C05_136EA', 'S0501_C05_136M', 'S0501_C05_136MA',
       'S0501_C05_137E', 'S0501_C05_137EA', 'S0501_C05_137M',
       'S0501_C05_137MA', 'combined statistical area', 'state_f', 'city_f'],
      dtype='object', length=2745)

In [614]:
def city_split(x):
    try:
        city = x.split("-")[0]
    except:
        pass
    return city
    
def state_split(x):
    try:
        state = x.split(" ")[-2]
        state = state.split("-")[0]
    except: 
        pass
    return state

fornav_char['state_f'] = fornav_char['NAME'].map(lambda x: state_split(x))

fornav_char['city_f'] = fornav_char['NAME'].map(lambda x: city_split(x))

In [615]:
fornav_char.head(3)

,NAME,GEO_ID,S0501_C01_001E,S0501_C01_001EA,S0501_C01_001M,S0501_C01_001MA,S0501_C01_002E,S0501_C01_002EA,S0501_C01_002M,S0501_C01_002MA,...,S0501_C05_136E,S0501_C05_136EA,S0501_C05_136M,S0501_C05_136MA,S0501_C05_137E,S0501_C05_137EA,S0501_C05_137M,S0501_C05_137MA,state_f,city_f
0,"Sacramento-Roseville, CA CSA",330M400US472,2619754,None,-555555555,*****,49.0,None,0.1,None,...,44.2,None,4.8,None,55.8,None,4.8,None,CA,Sacramento
1,"Saginaw-Midland-Bay City, MI CSA",330M400US474,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,MI,Saginaw
2,"St. Louis-St. Charles-Farmington, MO-IL CSA",330M400US476,2909036,None,2197,None,48.6,None,0.1,None,...,67.7,None,7.0,None,32.3,None,7.0,None,MO,St. Louis


In [616]:
fornav_char.columns

Index(['NAME', 'GEO_ID', 'S0501_C01_001E', 'S0501_C01_001EA', 'S0501_C01_001M',
       'S0501_C01_001MA', 'S0501_C01_002E', 'S0501_C01_002EA',
       'S0501_C01_002M', 'S0501_C01_002MA',
       ...
       'S0501_C05_136E', 'S0501_C05_136EA', 'S0501_C05_136M',
       'S0501_C05_136MA', 'S0501_C05_137E', 'S0501_C05_137EA',
       'S0501_C05_137M', 'S0501_C05_137MA', 'state_f', 'city_f'],
      dtype='object', length=2744)

In [611]:
fornav_char = fornav_char.drop(['combined statistical area'], axis=1)

In [617]:
 #Set variables
year='2018'
dsource='acs'
dname='acs1'
series = "subject"
cols='NAME,group(S0501)'
keyfile='census_key.txt'
geography="place"



In [618]:
fornav_char_pl = retreive_census(year, dsource,dname,cols,series,keyfile)
fornav_char_pl.head(3)

Data dumped to json file
File has 631 records


,NAME,GEO_ID,NAME,S0501_C01_001E,S0501_C01_001EA,S0501_C01_001M,S0501_C01_001MA,S0501_C01_002E,S0501_C01_002EA,S0501_C01_002M,...,S0501_C05_136E,S0501_C05_136EA,S0501_C05_136M,S0501_C05_136MA,S0501_C05_137E,S0501_C05_137EA,S0501_C05_137M,S0501_C05_137MA,state,place
0,"Phoenix city, Arizona",1600000US0455000,"Phoenix city, Arizona",1660272,None,81,None,49.7,None,0.4,...,54.2,None,5.0,None,45.8,None,5.0,None,04,55000
1,"Tucson city, Arizona",1600000US0477000,"Tucson city, Arizona",545987,None,59,None,49.2,None,0.7,...,55.2,None,9.3,None,44.8,None,9.3,None,04,77000
2,"Fresno city, California",1600000US0627000,"Fresno city, California",530073,None,81,None,49.4,None,0.6,...,43.4,None,8.3,None,56.6,None,8.3,None,06,27000


In [619]:
fornav_char_pl= pd.merge(fornav_char_pl, state_codes[[' st',' stusps']], how = 'left', left_on=["state"], right_on=[' st'])
fornav_char_pl = fornav_char_pl.drop(" st", axis=1)
fornav_char_pl.head(3)

,NAME,GEO_ID,NAME,S0501_C01_001E,S0501_C01_001EA,S0501_C01_001M,S0501_C01_001MA,S0501_C01_002E,S0501_C01_002EA,S0501_C01_002M,...,S0501_C05_136EA,S0501_C05_136M,S0501_C05_136MA,S0501_C05_137E,S0501_C05_137EA,S0501_C05_137M,S0501_C05_137MA,state,place,stusps
0,"Phoenix city, Arizona",1600000US0455000,"Phoenix city, Arizona",1660272,None,81,None,49.7,None,0.4,...,None,5.0,None,45.8,None,5.0,None,04,55000,AZ
1,"Tucson city, Arizona",1600000US0477000,"Tucson city, Arizona",545987,None,59,None,49.2,None,0.7,...,None,9.3,None,44.8,None,9.3,None,04,77000,AZ
2,"Fresno city, California",1600000US0627000,"Fresno city, California",530073,None,81,None,49.4,None,0.6,...,None,8.3,None,56.6,None,8.3,None,06,27000,CA


In [620]:
fornav_char_pl = fornav_char_pl.loc[:, ~fornav_char_pl.columns.duplicated()]
fornav_char_pl.columns

def city_split(x):
    city = x.split(" city")[0]
    return city

In [621]:
def city_split(x):
    city = x.split(" city")[0]
    return city
    
fornav_char_pl['city_f'] = fornav_char_pl['NAME'].map(lambda x: city_split(x))
fornav_char_pl = fornav_char_pl.rename(columns={' stusps': 'state_f'})
fornav_char_pl['state_f'] = fornav_char_pl['state_f'].str.strip()
fornav_char_pl.head(3)

,NAME,GEO_ID,S0501_C01_001E,S0501_C01_001EA,S0501_C01_001M,S0501_C01_001MA,S0501_C01_002E,S0501_C01_002EA,S0501_C01_002M,S0501_C01_002MA,...,S0501_C05_136M,S0501_C05_136MA,S0501_C05_137E,S0501_C05_137EA,S0501_C05_137M,S0501_C05_137MA,state,place,state_f,city_f
0,"Phoenix city, Arizona",1600000US0455000,1660272,None,81,None,49.7,None,0.4,None,...,5.0,None,45.8,None,5.0,None,04,55000,AZ,Phoenix
1,"Tucson city, Arizona",1600000US0477000,545987,None,59,None,49.2,None,0.7,None,...,9.3,None,44.8,None,9.3,None,04,77000,AZ,Tucson
2,"Fresno city, California",1600000US0627000,530073,None,81,None,49.4,None,0.6,None,...,8.3,None,56.6,None,8.3,None,06,27000,CA,Fresno


In [622]:
fornav_char_pl.columns

Index(['NAME', 'GEO_ID', 'S0501_C01_001E', 'S0501_C01_001EA', 'S0501_C01_001M',
       'S0501_C01_001MA', 'S0501_C01_002E', 'S0501_C01_002EA',
       'S0501_C01_002M', 'S0501_C01_002MA',
       ...
       'S0501_C05_136M', 'S0501_C05_136MA', 'S0501_C05_137E',
       'S0501_C05_137EA', 'S0501_C05_137M', 'S0501_C05_137MA', 'state',
       'place', 'state_f', 'city_f'],
      dtype='object', length=2746)

In [623]:
fornav_char_pl = fornav_char_pl.drop(['state','place'], axis=1)

In [624]:
#Set variables
year='2018'
dsource='acs'
dname='acs1'
series = "subject"
cols='NAME,group(S0501)'
keyfile='census_key.txt'
geography="metropolitan%20statistical%20area/micropolitan%20statistical%20area"

In [625]:
fornav_char_msa = retreive_census(year, dsource,dname,cols,series,keyfile)
fornav_char_msa.head(3)

Data dumped to json file
File has 520 records


,NAME,GEO_ID,NAME,S0501_C01_001E,S0501_C01_001EA,S0501_C01_001M,S0501_C01_001MA,S0501_C01_002E,S0501_C01_002EA,S0501_C01_002M,...,S0501_C05_135MA,S0501_C05_136E,S0501_C05_136EA,S0501_C05_136M,S0501_C05_136MA,S0501_C05_137E,S0501_C05_137EA,S0501_C05_137M,S0501_C05_137MA,metropolitan statistical area/micropolitan statistical area
0,"Albany-Schenectady-Troy, NY Metro Area",310M400US10580,"Albany-Schenectady-Troy, NY Metro Area",883169,None,-555555555,*****,49.1,None,0.2,...,None,68.3,None,10.5,None,31.7,None,10.5,None,10580
1,"Albuquerque, NM Metro Area",310M400US10740,"Albuquerque, NM Metro Area",916791,None,2319,None,49.4,None,0.2,...,None,50.1,None,11.6,None,49.9,None,11.6,None,10740
2,"Allentown-Bethlehem-Easton, PA-NJ Metro Area",310M400US10900,"Allentown-Bethlehem-Easton, PA-NJ Metro Area",842913,None,-555555555,*****,49.1,None,0.1,...,None,58.5,None,11.0,None,41.5,None,11.0,None,10900


In [626]:
fornav_char_msa = fornav_char_msa.loc[:, ~fornav_char_msa.columns.duplicated()]
fornav_char_msa.columns

Index(['NAME', 'GEO_ID', 'S0501_C01_001E', 'S0501_C01_001EA', 'S0501_C01_001M',
       'S0501_C01_001MA', 'S0501_C01_002E', 'S0501_C01_002EA',
       'S0501_C01_002M', 'S0501_C01_002MA',
       ...
       'S0501_C05_135MA', 'S0501_C05_136E', 'S0501_C05_136EA',
       'S0501_C05_136M', 'S0501_C05_136MA', 'S0501_C05_137E',
       'S0501_C05_137EA', 'S0501_C05_137M', 'S0501_C05_137MA',
       'metropolitan statistical area/micropolitan statistical area'],
      dtype='object', length=2743)

In [627]:
def city_split(x):
    city = x.split("-")[0]
    city = city.split(",")[0]
    return city
    
def state_split(x):
    state = x.split(",")[1]
    state = state.split(" ")[1]
    state = state.split("-")[0]
    return state
fornav_char_msa['state_f'] = fornav_char_msa['NAME'].map(lambda x: state_split(x))

fornav_char_msa['city_f'] = fornav_char_msa['NAME'].map(lambda x: city_split(x))

fornav_char_msa.head(10)

,NAME,GEO_ID,S0501_C01_001E,S0501_C01_001EA,S0501_C01_001M,S0501_C01_001MA,S0501_C01_002E,S0501_C01_002EA,S0501_C01_002M,S0501_C01_002MA,...,S0501_C05_136EA,S0501_C05_136M,S0501_C05_136MA,S0501_C05_137E,S0501_C05_137EA,S0501_C05_137M,S0501_C05_137MA,metropolitan statistical area/micropolitan statistical area,state_f,city_f
0,"Albany-Schenectady-Troy, NY Metro Area",310M400US10580,883169,None,-555555555,*****,49.1,None,0.2,None,...,None,10.5,None,31.7,None,10.5,None,10580,NY,Albany
1,"Albuquerque, NM Metro Area",310M400US10740,916791,None,2319,None,49.4,None,0.2,None,...,None,11.6,None,49.9,None,11.6,None,10740,NM,Albuquerque
2,"Allentown-Bethlehem-Easton, PA-NJ Metro Area",310M400US10900,842913,None,-555555555,*****,49.1,None,0.1,None,...,None,11.0,None,41.5,None,11.0,None,10900,PA,Allentown
3,"Atlanta-Sandy Springs-Roswell, GA Metro Area",310M400US12060,5950828,None,2546,None,48.4,None,0.1,None,...,None,3.3,None,46.6,None,3.3,None,12060,GA,Atlanta
4,"Austin-Round Rock, TX Metro Area",310M400US12420,2168316,None,-555555555,*****,50.1,None,0.1,None,...,None,5.0,None,52.0,None,5.0,None,12420,TX,Austin
5,"Bakersfield, CA Metro Area",310M400US12540,896764,None,-555555555,*****,51.3,None,0.1,None,...,None,5.7,None,59.7,None,5.7,None,12540,CA,Bakersfield
6,"Baltimore-Columbia-Towson, MD Metro Area",310M400US12580,2802789,None,-555555555,*****,48.2,None,0.1,None,...,None,6.5,None,45.5,None,6.5,None,12580,MD,Baltimore
7,"Boston-Cambridge-Newton, MA-NH Metro Area",310M400US14460,4875390,None,-555555555,*****,48.6,None,0.1,None,...,None,2.9,None,50.1,None,2.9,None,14460,MA,Boston
8,"Bridgeport-Stamford-Norwalk, CT Metro Area",310M400US14860,943823,None,-555555555,*****,48.7,None,0.1,None,...,None,5.5,None,59.0,None,5.5,None,14860,CT,Bridgeport
9,"Buffalo-Cheektowaga-Niagara Falls, NY Metro Area",310M400US15380,1130152,None,-555555555,*****,48.5,None,0.1,None,...,None,8.9,None,51.9,None,8.9,None,15380,NY,Buffalo


In [628]:
fornav_char_msa.columns

Index(['NAME', 'GEO_ID', 'S0501_C01_001E', 'S0501_C01_001EA', 'S0501_C01_001M',
       'S0501_C01_001MA', 'S0501_C01_002E', 'S0501_C01_002EA',
       'S0501_C01_002M', 'S0501_C01_002MA',
       ...
       'S0501_C05_136EA', 'S0501_C05_136M', 'S0501_C05_136MA',
       'S0501_C05_137E', 'S0501_C05_137EA', 'S0501_C05_137M',
       'S0501_C05_137MA',
       'metropolitan statistical area/micropolitan statistical area',
       'state_f', 'city_f'],
      dtype='object', length=2745)

In [629]:
fornav_char_msa = fornav_char_msa.drop(['metropolitan statistical area/micropolitan statistical area'], axis=1)

In [630]:
fornav_char = pd.concat([fornav_char,fornav_char_pl,fornav_char_msa], ignore_index=True, sort=False)

In [632]:
fornav_char = fornav_char.sort_values(['S0501_C03_101E']).drop_duplicates(['city_f', 'state_f'], keep='first')

In [640]:
fornav_char_sub = fornav_char[['NAME', 'city_f', 'state_f', 'GEO_ID', 'S0501_C03_001E', 'S0501_C03_013E', 'S0501_C03_026E', 'S0501_C03_027E', 'S0501_C03_033E', 'S0501_C03_034E', 'S0501_C03_035E', 'S0501_C03_036E', 'S0501_C03_037E', 'S0501_C03_039E', 'S0501_C03_040E', 'S0501_C03_041E', 'S0501_C03_042E', 'S0501_C03_043E', 'S0501_C03_044E', 'S0501_C03_045E', 'S0501_C03_046E', 'S0501_C03_047E', 'S0501_C03_048E', 'S0501_C03_049E', 'S0501_C03_050E', 'S0501_C03_051E', 'S0501_C03_052E', 'S0501_C03_055E', 'S0501_C03_057E', 'S0501_C03_061E', 'S0501_C03_062E', 'S0501_C03_063E', 'S0501_C03_064E', 'S0501_C03_065E', 'S0501_C03_066E', 'S0501_C03_067E', 'S0501_C03_068E', 'S0501_C03_069E', 'S0501_C03_070E', 'S0501_C03_071E', 'S0501_C03_072E', 'S0501_C03_073E', 'S0501_C03_074E', 'S0501_C03_075E', 'S0501_C03_076E', 'S0501_C03_077E', 'S0501_C03_078E', 'S0501_C03_079E', 'S0501_C03_089E', 'S0501_C03_090E', 'S0501_C03_091E', 'S0501_C03_097E', 'S0501_C03_101E', 'S0501_C03_103E', 'S0501_C03_116E', 'S0501_C03_117E', 'S0501_C03_118E', 'S0501_C03_128E', 'S0501_C03_131E', 'S0501_C03_132E', 'S0501_C03_135E', 'S0501_C03_136E', 'S0501_C03_137E', 'S0501_C04_001E', 'S0501_C05_001E', 'S0501_C05_013E', 'S0501_C05_026E', 'S0501_C05_027E', 'S0501_C05_033E', 'S0501_C05_034E', 'S0501_C05_035E', 'S0501_C05_036E', 'S0501_C05_037E', 'S0501_C05_039E', 'S0501_C05_040E', 'S0501_C05_041E', 'S0501_C05_042E', 'S0501_C05_043E', 'S0501_C05_044E', 'S0501_C05_046E', 'S0501_C05_047E', 'S0501_C05_048E', 'S0501_C05_049E', 'S0501_C05_050E', 'S0501_C05_051E', 'S0501_C05_052E', 'S0501_C05_053E', 'S0501_C05_054E', 'S0501_C05_055E', 'S0501_C05_056E', 'S0501_C05_057E', 'S0501_C05_058E', 'S0501_C05_059E', 'S0501_C05_061E', 'S0501_C05_062E', 'S0501_C05_063E', 'S0501_C05_064E', 'S0501_C05_065E', 'S0501_C05_066E', 'S0501_C05_067E', 'S0501_C05_068E', 'S0501_C05_069E', 'S0501_C05_070E', 'S0501_C05_071E', 'S0501_C05_072E', 'S0501_C05_073E', 'S0501_C05_074E', 'S0501_C05_075E', 'S0501_C05_076E', 'S0501_C05_077E', 'S0501_C05_078E', 'S0501_C05_079E', 'S0501_C05_089E', 'S0501_C05_090E', 'S0501_C05_091E', 'S0501_C05_096E', 'S0501_C05_101E','S0501_C05_103E', 'S0501_C05_116E', 'S0501_C05_117E', 'S0501_C05_118E', 'S0501_C05_128E']]

In [641]:
fornav_char_sub.head()

,NAME,city_f,state_f,GEO_ID,S0501_C03_001E,S0501_C03_013E,S0501_C03_026E,S0501_C03_027E,S0501_C03_033E,S0501_C03_034E,...,S0501_C05_089E,S0501_C05_090E,S0501_C05_091E,S0501_C05_096E,S0501_C05_101E,S0501_C05_103E,S0501_C05_116E,S0501_C05_117E,S0501_C05_118E,S0501_C05_128E
178,"San Jose city, California",San Jose,CA,1600000US0668000,407839,46.4,3.52,3.78,43105,2.0,...,58632,97.7,142735,1.5,116074,172509,58632,32.0,68.0,5.5
10,"San Juan-Carolina, PR CSA",San Juan,PR,330M400US490,79609,52.6,2.56,3.15,8676,1.3,...,15957,76.5,33159,3.1,15659,39456,15957,33.4,66.6,21.7
67,"McAllen-Edinburg, TX CSA",McAllen,TX,330M400US365,254226,45.7,4.02,4.53,24321,4.8,...,73031,82.2,42320,2.1,26468,185452,73031,62.6,37.4,9.8
155,"El Paso-Las Cruces, TX-NM CSA",El Paso,TX,330M400US238,249334,51.7,3.18,3.87,24744,0.2,...,53972,81.2,39654,5.9,27258,127343,53972,47.7,52.3,7.2
172,"Tucson city, Arizona",Tucson,AZ,1600000US0477000,86236,43.7,2.91,3.65,14928,0.8,...,18741,78.8,42765,3.8,32599,47100,18741,39.1,60.9,18.0


In [642]:
fornav_char_sub[fornav_char_sub['S0501_C03_001E'].notnull()]

,NAME,city_f,state_f,GEO_ID,S0501_C03_001E,S0501_C03_013E,S0501_C03_026E,S0501_C03_027E,S0501_C03_033E,S0501_C03_034E,...,S0501_C05_089E,S0501_C05_090E,S0501_C05_091E,S0501_C05_096E,S0501_C05_101E,S0501_C05_103E,S0501_C05_116E,S0501_C05_117E,S0501_C05_118E,S0501_C05_128E
178,"San Jose city, California",San Jose,CA,1600000US0668000,407839,46.4,3.52,3.78,43105,2.0,...,58632,97.7,142735,1.5,116074,172509,58632,32.0,68.0,5.5
10,"San Juan-Carolina, PR CSA",San Juan,PR,330M400US490,79609,52.6,2.56,3.15,8676,1.3,...,15957,76.5,33159,3.1,15659,39456,15957,33.4,66.6,21.7
67,"McAllen-Edinburg, TX CSA",McAllen,TX,330M400US365,254226,45.7,4.02,4.53,24321,4.8,...,73031,82.2,42320,2.1,26468,185452,73031,62.6,37.4,9.8
155,"El Paso-Las Cruces, TX-NM CSA",El Paso,TX,330M400US238,249334,51.7,3.18,3.87,24744,0.2,...,53972,81.2,39654,5.9,27258,127343,53972,47.7,52.3,7.2
172,"Tucson city, Arizona",Tucson,AZ,1600000US0477000,86236,43.7,2.91,3.65,14928,0.8,...,18741,78.8,42765,3.8,32599,47100,18741,39.1,60.9,18.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
180,"Washington city, District of Columbia",Washington,DC,1600000US1150000,97846,39.6,2.35,3.44,20895,1.3,...,19216,88.7,103522,-999999999.0,68831,50335,19216,23.8,76.2,50.3
868,"Urban Honolulu, HI Metro Area",Urban Honolulu,HI,310M400US46520,194237,50.1,3.57,4.19,21449,1.2,...,19285,84.5,78710,4.4,60162,72147,19285,27.6,72.4,25.6
807,"Baltimore-Columbia-Towson, MD Metro Area",Baltimore,MD,310M400US12580,290528,42.7,3.11,3.67,45916,0.5,...,42325,95.5,90190,0.7,64924,124622,42325,40.8,59.2,10.2
177,"San Francisco city, California",San Francisco,CA,1600000US0667000,295350,50.4,2.64,3.27,27726,1.0,...,40645,89.6,128016,3.6,82027,108633,40645,15.6,84.4,46.8


In [643]:
acs_groups = pd.read_html("https://api.census.gov/data/2018/acs/acs1/subject/variables.html")[0]
acs_groups.head()

,Name,Label,Concept,Required,Attributes,Limit,Predicate Type,Group,Unnamed: 8
0,AIANHH,Geography,NaN,not required,NaN,0,(not a predicate),NaN,NaN
1,ANRC,Geography,NaN,not required,NaN,0,(not a predicate),NaN,NaN
2,CBSA,Geography,NaN,not required,NaN,0,(not a predicate),NaN,NaN
3,CD116,Geography,NaN,not required,NaN,0,(not a predicate),NaN,NaN
4,CNECTA,Geography,NaN,not required,NaN,0,(not a predicate),NaN,NaN


In [644]:
cols = fornav_char_sub.columns.tolist()
id_cols= cols[0:4]
value_vars = cols[2:]

In [645]:
id_cols

['NAME', 'city_f', 'state_f', 'GEO_ID']

In [646]:
fornav_char_sub_long = fornav_char_sub.melt(id_vars=id_cols, value_vars=value_vars, var_name="Variable", value_name='Value', ignore_index=True)
fornav_char_sub_long.head()

,NAME,city_f,state_f,GEO_ID,Variable,Value
0,"San Jose city, California",San Jose,CA,1600000US0668000,S0501_C03_001E,407839
1,"San Juan-Carolina, PR CSA",San Juan,PR,330M400US490,S0501_C03_001E,79609
2,"McAllen-Edinburg, TX CSA",McAllen,TX,330M400US365,S0501_C03_001E,254226
3,"El Paso-Las Cruces, TX-NM CSA",El Paso,TX,330M400US238,S0501_C03_001E,249334
4,"Tucson city, Arizona",Tucson,AZ,1600000US0477000,S0501_C03_001E,86236


In [647]:
fornav_char_sub_long = pd.merge(fornav_char_sub_long, acs_groups[['Name','Label']], how = 'left', left_on=["Variable"], right_on=['Name'])
fornav_char_sub_long.head(10)

,NAME,city_f,state_f,GEO_ID,Variable,Value,Name,Label
0,"San Jose city, California",San Jose,CA,1600000US0668000,S0501_C03_001E,407839,S0501_C03_001E,Estimate!!Foreign born!!Total population
1,"San Juan-Carolina, PR CSA",San Juan,PR,330M400US490,S0501_C03_001E,79609,S0501_C03_001E,Estimate!!Foreign born!!Total population
2,"McAllen-Edinburg, TX CSA",McAllen,TX,330M400US365,S0501_C03_001E,254226,S0501_C03_001E,Estimate!!Foreign born!!Total population
3,"El Paso-Las Cruces, TX-NM CSA",El Paso,TX,330M400US238,S0501_C03_001E,249334,S0501_C03_001E,Estimate!!Foreign born!!Total population
4,"Tucson city, Arizona",Tucson,AZ,1600000US0477000,S0501_C03_001E,86236,S0501_C03_001E,Estimate!!Foreign born!!Total population
5,"Visalia-Porterville-Hanford, CA CSA",Visalia,CA,330M400US546,S0501_C03_001E,128656,S0501_C03_001E,Estimate!!Foreign born!!Total population
6,"Buffalo-Cheektowaga-Niagara Falls, NY Metro Area",Buffalo,NY,310M400US15380,S0501_C03_001E,81120,S0501_C03_001E,Estimate!!Foreign born!!Total population
7,"San Antonio city, Texas",San Antonio,TX,1600000US4865000,S0501_C03_001E,229219,S0501_C03_001E,Estimate!!Foreign born!!Total population
8,"Greensboro--Winston-Salem--High Point, NC CSA",Greensboro,NC,330M400US268,S0501_C03_001E,132335,S0501_C03_001E,Estimate!!Foreign born!!Total population
9,"Indianapolis city (balance), Indiana",Indianapolis,IN,1600000US1836003,S0501_C03_001E,81764,S0501_C03_001E,Estimate!!Foreign born!!Total population


In [648]:
#fornav_char_sub_long.to_csv('foreign_born_econ_codebook.csv', index=False, encoding = "utf_8")

In [649]:
#fornav_char_sub_long = fornav_char_sub_long.drop(['Name'], axis=1)

In [650]:
fornav_char_sub_long_codebook = fornav_char_sub_long[['Variable','Label']].drop_duplicates()

In [651]:
fornav_char_sub_long_codebook.head()

,Variable,Label
0,S0501_C03_001E,Estimate!!Foreign born!!Total population
934,S0501_C03_013E,Estimate!!Foreign born!!Total population!!SEX ...
1868,S0501_C03_026E,Estimate!!Foreign born!!Total population!!Aver...
2802,S0501_C03_027E,Estimate!!Foreign born!!Total population!!Aver...
3736,S0501_C03_033E,Estimate!!Foreign born!!SCHOOL ENROLLMENT!!Pop...


In [652]:
fornav_char_sub_long_codebook = fornav_char_sub_long_codebook.drop_duplicates(subset=['Variable'])

In [653]:
#fornav_char_sub_long_codebook.to_csv('foreignbornpops_codebook.csv', index=False, encoding = "utf_8")

In [654]:
fornav_char_sub_long = fornav_char_sub_long.drop_duplicates(subset=['city_f', 'state_f', 'Variable'], keep='first')

In [656]:
fornav_char_sub_final = fornav_char_sub_long.pivot(index=['NAME', 'city_f','state_f', 'GEO_ID'], columns='Label', values='Value').reset_index()
fornav_char_sub_final.head()

Label,NAME,city_f,state_f,GEO_ID,Estimate!!Foreign born!!Civilian employed population 16 years and over!!CLASS OF WORKER!!Private wage and salary workers,"Estimate!!Foreign born!!Civilian employed population 16 years and over!!INDUSTRY!!Agriculture, forestry, fishing and hunting, and mining","Estimate!!Foreign born!!Civilian employed population 16 years and over!!INDUSTRY!!Arts, entertainment, and recreation, and accommodation and food services",Estimate!!Foreign born!!Civilian employed population 16 years and over!!INDUSTRY!!Construction,"Estimate!!Foreign born!!Civilian employed population 16 years and over!!INDUSTRY!!Educational services, and health care and social assistance","Estimate!!Foreign born!!Civilian employed population 16 years and over!!INDUSTRY!!Finance and insurance, and real estate and rental and leasing",...,Estimate!!Foreign born; Not a U.S. citizen!!POVERTY STATUS IN THE PAST 12 MONTHS!!Population for whom poverty status is determined,Estimate!!Foreign born; Not a U.S. citizen!!SCHOOL ENROLLMENT!!Population 3 years and over enrolled in school,Estimate!!Foreign born; Not a U.S. citizen!!SCHOOL ENROLLMENT!!Population 3 years and over enrolled in school!!College or graduate school,Estimate!!Foreign born; Not a U.S. citizen!!SCHOOL ENROLLMENT!!Population 3 years and over enrolled in school!!Elementary school (grades K-8),Estimate!!Foreign born; Not a U.S. citizen!!SCHOOL ENROLLMENT!!Population 3 years and over enrolled in school!!High school (grades 9-12),"Estimate!!Foreign born; Not a U.S. citizen!!SCHOOL ENROLLMENT!!Population 3 years and over enrolled in school!!Nursery school, preschool",Estimate!!Foreign born; Not a U.S. citizen!!Total population,Estimate!!Foreign born; Not a U.S. citizen!!Total population!!Average family size,Estimate!!Foreign born; Not a U.S. citizen!!Total population!!Average household size,Estimate!!Foreign born; Not a U.S. citizen!!Total population!!SEX AND AGE!!Median age (years)
0,"Aberdeen, WA Micro Area",Aberdeen,WA,310M400US10140,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
1,"Abilene city, Texas",Abilene,TX,1600000US4801000,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
2,"Adrian, MI Micro Area",Adrian,MI,310M400US10300,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
3,"Aguadilla-Isabela, PR Metro Area",Aguadilla,PR,310M400US10380,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
4,"Akron city, Ohio",Akron,OH,1600000US3901000,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None


In [657]:
fornav_char_sub_final['FY'] = "2018"

In [659]:
fornav_char_sub_final = fornav_char_sub_final.drop_duplicates(subset=['city_f','state_f','FY'], keep="first")

In [661]:
fornav_char_sub_final['city_f'].iloc[1]

'Abilene'

In [662]:
all_cities_vouchfoodaff_econ_geo = pd.merge(all_cities_vouchfoodaff_geo, fornav_char_sub_final, how = 'left', left_on=['city', 'STATE', "year"], right_on=['city_f', 'state_f','FY'])
all_cities_vouchfoodaff_econ_geo.head(10)

,state,year,statetotal,origin,stateorigin,city,cityorigin,annualtotal,Region of Origin,Region Refugee Admissions Ceiling,...,Estimate!!Foreign born; Not a U.S. citizen!!SCHOOL ENROLLMENT!!Population 3 years and over enrolled in school,Estimate!!Foreign born; Not a U.S. citizen!!SCHOOL ENROLLMENT!!Population 3 years and over enrolled in school!!College or graduate school,Estimate!!Foreign born; Not a U.S. citizen!!SCHOOL ENROLLMENT!!Population 3 years and over enrolled in school!!Elementary school (grades K-8),Estimate!!Foreign born; Not a U.S. citizen!!SCHOOL ENROLLMENT!!Population 3 years and over enrolled in school!!High school (grades 9-12),"Estimate!!Foreign born; Not a U.S. citizen!!SCHOOL ENROLLMENT!!Population 3 years and over enrolled in school!!Nursery school, preschool",Estimate!!Foreign born; Not a U.S. citizen!!Total population,Estimate!!Foreign born; Not a U.S. citizen!!Total population!!Average family size,Estimate!!Foreign born; Not a U.S. citizen!!Total population!!Average household size,Estimate!!Foreign born; Not a U.S. citizen!!Total population!!SEX AND AGE!!Median age (years),FY_y
0,Alabama,2016,120,Afghanistan,9,Mobile,9,84994,Near East/South Asia,38000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Alabama,2017,63,Afghanistan,9,Mobile,9,53716,Near East/South Asia,40000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Alabama,2018,38,Afghanistan,0,Mobile,0,22491,Near East/South Asia,17500.0,...,None,None,None,None,None,None,None,None,None,2018
3,Alabama,2016,120,Burma,3,Auburn,0,84994,East Asia,14000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Alabama,2017,63,Burma,1,Auburn,0,53716,East Asia,12000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Alabama,2018,38,Burma,1,Auburn,0,22491,East Asia,5000.0,...,None,None,None,None,None,None,None,None,None,2018
6,Alabama,2016,120,Burma,3,Birmingham,0,84994,East Asia,14000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,Alabama,2017,63,Burma,1,Birmingham,1,53716,East Asia,12000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,Alabama,2018,38,Burma,1,Birmingham,1,22491,East Asia,5000.0,...,None,None,None,None,None,None,None,None,None,2018
9,Alabama,2016,120,Burma,3,Dothan,0,84994,East Asia,14000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [663]:
#all_cities_vouchfoodaff_econ_geo.to_csv('full_cities_data.csv', index=False, encoding = "utf_8_sig")


In [664]:
all_cities_vouchfoodaff_econ_geo_2018 = all_cities_vouchfoodaff_econ_geo[all_cities_vouchfoodaff_econ_geo['year'] == "2018"]
all_cities_vouchfoodaff_econ_geo.head()

,state,year,statetotal,origin,stateorigin,city,cityorigin,annualtotal,Region of Origin,Region Refugee Admissions Ceiling,...,Estimate!!Foreign born; Not a U.S. citizen!!SCHOOL ENROLLMENT!!Population 3 years and over enrolled in school,Estimate!!Foreign born; Not a U.S. citizen!!SCHOOL ENROLLMENT!!Population 3 years and over enrolled in school!!College or graduate school,Estimate!!Foreign born; Not a U.S. citizen!!SCHOOL ENROLLMENT!!Population 3 years and over enrolled in school!!Elementary school (grades K-8),Estimate!!Foreign born; Not a U.S. citizen!!SCHOOL ENROLLMENT!!Population 3 years and over enrolled in school!!High school (grades 9-12),"Estimate!!Foreign born; Not a U.S. citizen!!SCHOOL ENROLLMENT!!Population 3 years and over enrolled in school!!Nursery school, preschool",Estimate!!Foreign born; Not a U.S. citizen!!Total population,Estimate!!Foreign born; Not a U.S. citizen!!Total population!!Average family size,Estimate!!Foreign born; Not a U.S. citizen!!Total population!!Average household size,Estimate!!Foreign born; Not a U.S. citizen!!Total population!!SEX AND AGE!!Median age (years),FY_y
0,Alabama,2016,120,Afghanistan,9,Mobile,9,84994,Near East/South Asia,38000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Alabama,2017,63,Afghanistan,9,Mobile,9,53716,Near East/South Asia,40000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Alabama,2018,38,Afghanistan,0,Mobile,0,22491,Near East/South Asia,17500.0,...,None,None,None,None,None,None,None,None,None,2018
3,Alabama,2016,120,Burma,3,Auburn,0,84994,East Asia,14000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Alabama,2017,63,Burma,1,Auburn,0,53716,East Asia,12000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [665]:
all_cities_vouchfoodaff_econ_geo.columns

Index(['state', 'year', 'statetotal', 'origin', 'stateorigin', 'city',
       'cityorigin', 'annualtotal', 'Region of Origin',
       'Region Refugee Admissions Ceiling',
       ...
       'Estimate!!Foreign born; Not a U.S. citizen!!SCHOOL ENROLLMENT!!Population 3 years and over enrolled in school',
       'Estimate!!Foreign born; Not a U.S. citizen!!SCHOOL ENROLLMENT!!Population 3 years and over enrolled in school!!College or graduate school',
       'Estimate!!Foreign born; Not a U.S. citizen!!SCHOOL ENROLLMENT!!Population 3 years and over enrolled in school!!Elementary school (grades K-8)',
       'Estimate!!Foreign born; Not a U.S. citizen!!SCHOOL ENROLLMENT!!Population 3 years and over enrolled in school!!High school (grades 9-12)',
       'Estimate!!Foreign born; Not a U.S. citizen!!SCHOOL ENROLLMENT!!Population 3 years and over enrolled in school!!Nursery school, preschool',
       'Estimate!!Foreign born; Not a U.S. citizen!!Total population',
       'Estimate!!Foreign born; N

In [667]:
all_cities_vouchfoodaff_econ_geo_2018[all_cities_vouchfoodaff_econ_geo_2018['value'] > 4772]['value'].describe()

count        45.000000
mean      42503.200000
std       86266.975964
min        4807.000000
25%       11396.000000
50%       18360.000000
75%       30997.000000
max      413088.000000
Name: value, dtype: float64

## Checks

In [668]:
fornav_char_sub_final[fornav_char_sub_final['Estimate!!Foreign born!!INCOME IN THE PAST 12 MONTHS (IN 2018 INFLATION-ADJUSTED DOLLARS)!!Households!!Median Household income (dollars)'].notnull()][['NAME', 'Estimate!!Foreign born!!INCOME IN THE PAST 12 MONTHS (IN 2018 INFLATION-ADJUSTED DOLLARS)!!Households!!Median Household income (dollars)']]['NAME'].value_counts()

Albany-Schenectady, NY CSA                       1
Modesto, CA Metro Area                           1
Richmond, VA Metro Area                          1
Reno-Carson City-Fernley, NV CSA                 1
Raleigh-Durham-Chapel Hill, NC CSA               1
                                                ..
Greenville-Spartanburg-Anderson, SC CSA          1
Greensboro--Winston-Salem--High Point, NC CSA    1
Grand Rapids-Wyoming-Muskegon, MI CSA            1
Fresno, CA Metro Area                            1
Worcester, MA-CT Metro Area                      1
Name: NAME, Length: 81, dtype: int64

In [670]:
all_cities_vouchfoodaff_econ_geo_2018[all_cities_vouchfoodaff_econ_geo_2018['Estimate!!Foreign born!!INCOME IN THE PAST 12 MONTHS (IN 2018 INFLATION-ADJUSTED DOLLARS)!!Households!!Median Household income (dollars)'].notnull()][['NAME', 'city', 'state','Estimate!!Foreign born!!INCOME IN THE PAST 12 MONTHS (IN 2018 INFLATION-ADJUSTED DOLLARS)!!Households!!Median Household income (dollars)']]['city'].value_counts()

Atlanta       74
Austin        60
Denver        53
Chicago       48
Phoenix       47
              ..
Cape Coral     3
Santa Rosa     3
Detroit        2
Oxnard         2
Visalia        1
Name: city, Length: 75, dtype: int64

In [674]:
all_cities_vouchfoodaff_econ_geo_2018.to_csv('full_cities_data_2018.csv', index=False, encoding = "utf_8")

In [677]:
pd.set_option('display.max_rows', None)
for i in all_cities_vouchfoodaff_econ_geo_2018.columns:
    print(i)

state
year
statetotal
origin
stateorigin
city
cityorigin
annualtotal
Region of Origin
Region Refugee Admissions Ceiling
ref_num
st
 stusps_x
AREA
PERIMETER
CITIESX020
FEATURE
FIPS
FIPS55
POP
STATE_FIPS
STATE
COUNTY
geometry
for_tiger
Census_Name
GEO_ID_x
Variable
value
Variable_Type
LA1and10
LA1and20
LAhalfand10
lahunv1
lahunv10
lahunv10share
lahunv20
lahunvhalf
lakids1
lakids10
lakids20
lakidshalf
lalowi1
lalowi10
lalowi20
lalowihalf
lapop1
lapop10
lapop20
lapophalf
laseniors1
laseniors10
laseniors20
laseniorshalf
LATracts_half
LATracts1
LATracts10
LATracts20
LILATracts_1And10
LILATracts_1And20
LILATracts_halfAnd10
LILATracts_Vehicle
LowIncomeTracts
NUMGQTRS
OHU2010
POP2010
Rural
lahunv1share
lahunv20share
lahunvhalfshare
lakids10share
lakids1share
lakids20share
lakidshalfshare
lalowi10share
lalowi1share
lalowi20share
lalowihalfshare
lapop10share
lapop1share
lapop20share
lapophalfshare
laseniors10share
laseniors1share
laseniors20share
laseniorshalfshare
PCTGQTRS
farm_mark_count
City
S